In [10]:
from itertools import islice
import time
from functions import relative_success, spreadSheetFormatter, convertTuple
#from array import ArrayType
from functions import (
    relative_success,
    spreadSheetFormatter,
    discrepancifier,
    temporary_directionOfEffect,
)
from functions import relative_success, spreadSheetFormatter, convertTuple
import re
import pandas as pd
import numpy as np
from scipy.stats import fisher_exact
from scipy.stats.contingency import odds_ratio, relative_risk
# from stoppedTrials import terminated_td
from DoEAssessment import directionOfEffect
# from membraneTargets import target_membrane
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F
#from itertools import islice
from datetime import datetime
from datetime import date
from pyspark.sql.types import (
    StructType,
    StructField,
    DoubleType,
    StringType,
    IntegerType,
    ArrayType
)
import pandas as pd
import gcsfs


from pyspark.sql import SparkSession
import numpy as np
spark = SparkSession.builder.getOrCreate()

full_data = spark.createDataFrame(
    data=[
        ("yes", "yes"),
        ("yes", "no"),
        ("no", "yes"),
        ("no", "no"),
    ],
    schema=StructType(
        [
            StructField("prediction", StringType(), True),
            StructField("comparison", StringType(), True),
        ]
    ),
)
results=[]
result = []
result_st = []
result_ci = []
array2 = []
results = []


fs = gcsfs.GCSFileSystem()  # Initialize Google Cloud Storage filesystem

folder_path = "gs://ot-team/jroldan/2025-05-06_analysis/*/"
parquet_files = fs.glob(f"{folder_path}*.parquet")

for path in parquet_files:
    path = 'gs://' + path
    print(f"Reading {path}")
    df = spark.read.parquet(path)
    array1 = np.delete(
        df.join(full_data, on=["prediction", "comparison"], how="outer")
        .groupBy("comparison")
        .pivot("prediction")
        .agg(F.first("a"))
        .sort(F.col("comparison").desc())
        .select("comparison", "yes", "no")
        .fillna(0)
        .toPandas()
        .to_numpy(),
        [0],
        1,
    )
    total = np.sum(array1)
    res_npPhaseX = np.array(array1, dtype=int)
    resX = convertTuple(fisher_exact(res_npPhaseX, alternative="two-sided"))
    resx_CI = convertTuple(
        odds_ratio(res_npPhaseX).confidence_interval(confidence_level=0.95)
    )

    result_st.append(resX)
    result_ci.append(resx_CI)
    (rs_result, rs_ci) = relative_success(array1)
    import re

    # 1. Extract the element between 'analysis/' and the next '/'
    match1 = re.search(r'analysis/([^/]+)/', path)
    part1 = match1.group(1) if match1 else None

    # 2. Extract the element after the second '/' until '_Phase'
    match2 = re.search(r'[^/]+/([^/_]+)_Phase', path)
    part2 = match2.group(1) if match2 else None

    # 3. Extract the element from 'Phase' until '.parquet'
    match3 = re.search(r'(Phase[^.]+)\.parquet', path)
    part3 = match3.group(1) if match3 else None

    print("Part 1:", part1)
    print("Part 2:", part2)
    print("Part 3:", part3)

    results.append(
        [
            part1, #data,
            part2, #comparisonColumn,
            part3, #predictionColumn,
            round(float(resX.split(",")[0]), 2),
            float(resX.split(",")[1]),
            round(float(resx_CI.split(",")[0]), 2),
            round(float(resx_CI.split(",")[1]), 2),
            str(total),
            np.array(res_npPhaseX).tolist(),
            round(float(rs_result), 2),
            round(float(rs_ci[0]), 2),
            round(float(rs_ci[1]), 2),
            path,
        ]
    )
    print('finished')

Reading gs://ot-team/jroldan/2025-05-06_analysis/df_WOcgc_All_original/NdiagonalYes_Phase4.parquet


Part 1: df_WOcgc_All_original
Part 2: NdiagonalYes
Part 3: Phase4
finished
Reading gs://ot-team/jroldan/2025-05-06_analysis/df_WOcgc_All_original/NdiagonalYes_Phase>=1.parquet
Part 1: df_WOcgc_All_original
Part 2: NdiagonalYes
Part 3: Phase>=1
finished
Reading gs://ot-team/jroldan/2025-05-06_analysis/df_WOcgc_All_original/NdiagonalYes_Phase>=2.parquet


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/miniconda3/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [9]:
fs = gcsfs.GCSFileSystem()  # Initialize Google Cloud Storage filesystem

folder_path = "gs://ot-team/jroldan/2025-05-06_analysis/*/"
parquet_files = fs.glob(f"{folder_path}*.parquet")

for path in parquet_files:
    print(path)
    path = 'gs://' + path
    print(path)

ot-team/jroldan/2025-05-06_analysis/df_WOcgc_All_original/NdiagonalYes_Phase4.parquet
gs://ot-team/jroldan/2025-05-06_analysis/df_WOcgc_All_original/NdiagonalYes_Phase4.parquet
ot-team/jroldan/2025-05-06_analysis/df_WOcgc_All_original/NdiagonalYes_Phase>=1.parquet
gs://ot-team/jroldan/2025-05-06_analysis/df_WOcgc_All_original/NdiagonalYes_Phase>=1.parquet
ot-team/jroldan/2025-05-06_analysis/df_WOcgc_All_original/NdiagonalYes_Phase>=2.parquet
gs://ot-team/jroldan/2025-05-06_analysis/df_WOcgc_All_original/NdiagonalYes_Phase>=2.parquet
ot-team/jroldan/2025-05-06_analysis/df_WOcgc_All_original/NdiagonalYes_Phase>=3.parquet
gs://ot-team/jroldan/2025-05-06_analysis/df_WOcgc_All_original/NdiagonalYes_Phase>=3.parquet
ot-team/jroldan/2025-05-06_analysis/df_WOcgc_All_original/NdiagonalYes_PhaseT.parquet
gs://ot-team/jroldan/2025-05-06_analysis/df_WOcgc_All_original/NdiagonalYes_PhaseT.parquet
ot-team/jroldan/2025-05-06_analysis/df_WOcgc_All_original/NoneCellYes_Phase4.parquet
gs://ot-team/jrold

In [6]:
path

'ot-team/jroldan/2025-05-06_analysis/df_WOcgc_All_original/NdiagonalYes_Phase4.parquet'

In [8]:
parquet_files

[]

In [1]:
from itertools import islice
import time
from functions import relative_success, spreadSheetFormatter, convertTuple
#from array import ArrayType
from functions import (
    relative_success,
    spreadSheetFormatter,
    discrepancifier,
    temporary_directionOfEffect,
)
from functions import relative_success, spreadSheetFormatter, convertTuple
import re
import pandas as pd
import numpy as np
from scipy.stats import fisher_exact
from scipy.stats.contingency import odds_ratio, relative_risk
# from stoppedTrials import terminated_td
from DoEAssessment import directionOfEffect
# from membraneTargets import target_membrane
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F
#from itertools import islice
from datetime import datetime
from datetime import date
from pyspark.sql.types import (
    StructType,
    StructField,
    DoubleType,
    StringType,
    IntegerType,
    ArrayType
)
import pandas as pd
import gcsfs


from pyspark.sql import SparkSession
import numpy as np
spark = SparkSession.builder.getOrCreate()

full_data = spark.createDataFrame(
    data=[
        ("yes", "yes"),
        ("yes", "no"),
        ("no", "yes"),
        ("no", "no"),
    ],
    schema=StructType(
        [
            StructField("prediction", StringType(), True),
            StructField("comparison", StringType(), True),
        ]
    ),
)
results=[]
result = []
result_st = []
result_ci = []
array2 = []
results = []

today_date = str(date.today())

fs = gcsfs.GCSFileSystem()  # Initialize Google Cloud Storage filesystem

folder_path = "gs://ot-team/jroldan/2025-05-07_analysis/*/"
parquet_files = fs.glob(f"{folder_path}*.parquet")

for path in parquet_files:
    path = 'gs://' + path
    print(f"Reading {path}")
    df = spark.read.parquet(path)
    array1 = np.delete(
        df.join(full_data, on=["prediction", "comparison"], how="outer")
        .groupBy("comparison")
        .pivot("prediction")
        .agg(F.first("a"))
        .sort(F.col("comparison").desc())
        .select("comparison", "yes", "no")
        .fillna(0)
        .toPandas()
        .to_numpy(),
        [0],
        1,
    )
    total = np.sum(array1)
    res_npPhaseX = np.array(array1, dtype=int)
    resX = convertTuple(fisher_exact(res_npPhaseX, alternative="two-sided"))
    resx_CI = convertTuple(
        odds_ratio(res_npPhaseX).confidence_interval(confidence_level=0.95)
    )

    result_st.append(resX)
    result_ci.append(resx_CI)
    (rs_result, rs_ci) = relative_success(array1)
    import re

    # 1. Extract the element between 'analysis/' and the next '/'
    match1 = re.search(r'analysis/([^/]+)/', path)
    part1 = match1.group(1) if match1 else None

    # 2. Extract the element after the second '/' until '_Phase'
    match2 = re.search(r'[^/]+/([^/_]+)_Phase', path)
    part2 = match2.group(1) if match2 else None

    # 3. Extract the element from 'Phase' until '.parquet'
    match3 = re.search(r'(Phase[^.]+)\.parquet', path)
    part3 = match3.group(1) if match3 else None

    print("Part 1:", part1)
    print("Part 2:", part2)
    print("Part 3:", part3)

    results.append(
        [
            part1, #data,
            part2, #comparisonColumn,
            part3, #predictionColumn,
            round(float(resX.split(",")[0]), 2),
            float(resX.split(",")[1]),
            round(float(resx_CI.split(",")[0]), 2),
            round(float(resx_CI.split(",")[1]), 2),
            str(total),
            np.array(res_npPhaseX).tolist(),
            round(float(rs_result), 2),
            round(float(rs_ci[0]), 2),
            round(float(rs_ci[1]), 2),
            path,
        ]
    )
    print('finished')

df_results = pd.DataFrame(
    results,
    columns=[
        "group",
        "comparison",
        "phase",
        "OR",
        "pValue",
        "LowCI",
        "HighCI",
        "total",
        "array",
        "rs",
        "lowRs",
        "HighRs",
        "path",
    ],
)
print("created pandas dataframe")
print("converting to spark dataframe")
print("preparing dataframe")

schema = StructType(
    [
        StructField("group", StringType(), True),
        StructField("comparison", StringType(), True),
        StructField("phase", StringType(), True),
        StructField("oddsRatio", DoubleType(), True),
        StructField("pValue", DoubleType(), True),
        StructField("lowerInterval", DoubleType(), True),
        StructField("upperInterval", DoubleType(), True),
        StructField("total", StringType(), True),
        StructField("values", ArrayType(ArrayType(IntegerType())), True),
        StructField("relSuccess", DoubleType(), True),
        StructField("rsLower", DoubleType(), True),
        StructField("rsUpper", DoubleType(), True),
        StructField("path", StringType(), True),
    ]
)

print("read pattern variables")
df = spreadSheetFormatter(spark.createDataFrame(df_results, schema=schema))
print("processed spreadsheet")
print("writting the dataframe")

# Convert list of lists to DataFrame
# Regular expressions
'''
value_pattern = r"df_([^_]+)_"  # Extracts {value}
middle_pattern = r"df_[^_]+_([^_]+)_"  # Extracts middle part (All, Other, etc.)
suffix_pattern = r"(original|propag)$"  # Extracts suffix (original or propag)
'''

df.withColumn(
    "datasource",
    F.regexp_extract(F.col("group"), r"df_(.*?)_(All|Other|OtherNull|Oncology)_(propag|original)", 1)
).withColumn(
    "therArea",
    F.regexp_extract(F.col("group"), r"_(All|Other|OtherNull|Oncology)_", 1)
).withColumn(
    "type",
    F.regexp_extract(F.col("group"), r"_(propag|original)$", 1)
).toPandas().to_csv(
    f"gs://ot-team/jroldan/analysis/{today_date}_genEvidAnalysis_new.csv"
)

print("dataframe written \n Analysis finished")

spark session created at 2025-05-07 15:28:47.619207
Analysis started on 2025-05-07 at  2025-05-07 15:28:47.619207


25/05/07 15:28:52 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Reading gs://ot-team/jroldan/2025-05-07_analysis/df_WOcgc_All_original/NdiagonalYes_Phase4.parquet


Part 1: df_WOcgc_All_original
Part 2: NdiagonalYes
Part 3: Phase4
finished
Reading gs://ot-team/jroldan/2025-05-07_analysis/df_WOcgc_All_original/NdiagonalYes_Phase>=1.parquet
Part 1: df_WOcgc_All_original
Part 2: NdiagonalYes
Part 3: Phase>=1
finished
Reading gs://ot-team/jroldan/2025-05-07_analysis/df_WOcgc_All_original/NdiagonalYes_Phase>=2.parquet
Part 1: df_WOcgc_All_original
Part 2: NdiagonalYes
Part 3: Phase>=2
finished
Reading gs://ot-team/jroldan/2025-05-07_analysis/df_WOcgc_All_original/NdiagonalYes_Phase>=3.parquet
Part 1: df_WOcgc_All_original
Part 2: NdiagonalYes
Part 3: Phase>=3
finished
Reading gs://ot-team/jroldan/2025-05-07_analysis/df_WOcgc_All_original/NdiagonalYes_PhaseT.parquet
Part 1: df_WOcgc_All_original
Part 2: NdiagonalYes
Part 3: PhaseT
finished
Reading gs://ot-team/jroldan/2025-05-07_analysis/df_WOcgc_All_original/NoneCellYes_Phase4.parquet
Part 1: df_WOcgc_All_original
Part 2: NoneCellYes
Part 3: Phase4
finished
Reading gs://ot-team/jroldan/2025-05-07_analy

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/miniconda3/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [3]:
import time
#from array import ArrayType
from functions import (
    discrepancifier,
    temporary_directionOfEffect,
)
# from stoppedTrials import terminated_td
from DoEAssessment import directionOfEffect
# from membraneTargets import target_membrane
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F
#from itertools import islice
from datetime import datetime
from datetime import date
from pyspark.sql.types import (
    StructType,
    StructField,
    DoubleType,
    StringType,
    IntegerType,
    ArrayType
)
import pandas as pd

spark = SparkSession.builder.getOrCreate()

platform_v = "25.03"

doe_sources = [
    "gwas_credible_set",
    "gene_burden",
    "eva",
    "eva_somatic",
    "gene2phenotype",
    "orphanet",
    "cancer_gene_census",
    "intogen",
    "impc",
    "chembl",
]

### ingest evidences and create gwas_credible_set evidences. 

path_n='gs://open-targets-data-releases/25.03/output/'

target = spark.read.parquet(f"{path_n}target/")

diseases = spark.read.parquet(f"{path_n}disease/")

evidences = spark.read.parquet(f"{path_n}evidence")

credible = spark.read.parquet(f"{path_n}credible_set")

new = spark.read.parquet(f"{path_n}colocalisation_coloc") 

index=spark.read.parquet(f"{path_n}study/")

variantIndex = spark.read.parquet(f"{path_n}variant")

biosample = spark.read.parquet(f"{path_n}biosample")

print("loaded files")

newColoc = (
    new.join(
        credible.selectExpr(  #### studyLocusId from credible set to uncover the codified variants on left side
            "studyLocusId as leftStudyLocusId",
            "StudyId as leftStudyId",
            "variantId as leftVariantId",
            "studyType as credibleLeftStudyType",
        ),
        on="leftStudyLocusId",
        how="left",
    )
    .join(
        credible.selectExpr(  #### studyLocusId from credible set to uncover the codified variants on right side
            "studyLocusId as rightStudyLocusId",
            "studyId as rightStudyId",
            "variantId as rightVariantId",
            "studyType as credibleRightStudyType",
            'isTransQtl'
        ),
        on="rightStudyLocusId",
        how="left",
    )
    .join(
        index.selectExpr(  ### bring modulated target on right side (QTL study)
            "studyId as rightStudyId",
            "geneId",
            "projectId",
            "studyType as indexStudyType",
            "condition",
            "biosampleId",
        ),
        on="rightStudyId",
        how="left",
)
    # .persist()
)

print("loaded newColoc")

# remove columns without content (only null values on them)
df = evidences.filter((F.col("datasourceId") == "gwas_credible_sets"))

# Use an aggregation to determine non-null columns
non_null_counts = df.select(
    *[F.sum(F.col(col).isNotNull().cast("int")).alias(col) for col in df.columns]
)

# Collect the counts for each column
non_null_columns = [
    row[0] for row in non_null_counts.collect()[0].asDict().items() if row[1] > 0
]

# Select only the non-null columns
filtered_df = df.select(*non_null_columns)  # .persist()

## bring studyId, variantId, beta from Gwas and pValue
gwasComplete = filtered_df.join(
    credible.selectExpr(
        "studyLocusId", "studyId", "variantId", "beta as betaGwas", "pValueExponent"
    ),
    on="studyLocusId",
    how="left",
)  # .persist()

print("loaded gwasComplete")

resolvedColoc = (
    (
        newColoc.withColumnRenamed("geneId", "targetId")
        .join(
            gwasComplete.withColumnRenamed("studyLocusId", "leftStudyLocusId"),
            on=["leftStudyLocusId", "targetId"],
            how="inner",
        )
        .join(  ### propagated using parent terms
            diseases.selectExpr(
                "id as diseaseId", "name", "parents", "therapeuticAreas"
            ),
            on="diseaseId",
            how="left",
        )
        .withColumn(
            "diseaseId",
            F.explode_outer(F.concat(F.array(F.col("diseaseId")), F.col("parents"))),
        )
        .drop("parents", "oldDiseaseId")
    ).withColumn(
        "colocDoE",
        F.when(
            F.col("rightStudyType").isin(
                ["eqtl", "pqtl", "tuqtl", "sceqtl", "sctuqtl"]
            ),
            F.when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("GoF_risk"),
            )
            .when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("LoF_risk"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("LoF_protect"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("GoF_protect"),
            ),
        ).when(
            F.col("rightStudyType").isin(
                ["sqtl", "scsqtl"]
            ),  ### opposite directionality than sqtl
            F.when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("LoF_risk"),
            )
            .when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("GoF_risk"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("GoF_protect"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("LoF_protect"),
            ),
        ),
    )
    # .persist()
)
print("loaded resolvedColloc")

datasource_filter = [
    #"gwas_credible_set", remove so avoid potential duplicates as it will be incorporated later (DoE is done separately)
    "gene_burden",
    "eva",
    "eva_somatic",
    "gene2phenotype",
    "orphanet",
    "cancer_gene_census",
    "intogen",
    "impc",
    "chembl",
]

assessment, evidences, actionType, oncolabel = temporary_directionOfEffect(
    path_n, datasource_filter
)

print("run temporary direction of effect")

window_spec = Window.partitionBy("targetId", "diseaseId",'leftStudyId').orderBy( ### include gwas study
    F.col("pValueExponent").asc()
)
gwasCredibleAssoc = (
    resolvedColoc.withColumn(
        "homogenized", F.first("colocDoE", ignorenulls=True).over(window_spec)
    )  ## added 30.01.2025
    .select("targetId", "diseaseId",'leftStudyId', "homogenized")
    .withColumn(
        "homogenized",
        F.when(F.col("homogenized").isNull(), F.lit("noEvaluable")).otherwise(
            F.col("homogenized")
        ),
    )
)

print("Moving to step 2")

columns_chembl = ["LoF_protect", "GoF_protect"]
columns_dataset = ["LoF_protect", "GoF_protect", "LoF_risk", "GoF_risk", "evidenceDif"]
columns = ["GoF_risk", "LoF_protect", "LoF_risk", "GoF_protect"]
terms = ["noEvaluable", "bivalent_risk", "null", "dispar"]

taDf = spark.createDataFrame(
    data=[
        ("MONDO_0045024", "cell proliferation disorder", "Oncology"),
        ("EFO_0005741", "infectious disease", "Other"),
        ("OTAR_0000014", "pregnancy or perinatal disease", "Other"),
        ("EFO_0005932", "animal disease", "Other"),
        ("MONDO_0024458", "disease of visual system", "Other"),
        ("EFO_0000319", "cardiovascular disease", "Other"),
        ("EFO_0009605", "pancreas disease", "Other"),
        ("EFO_0010282", "gastrointestinal disease", "Other"),
        ("OTAR_0000017", "reproductive system or breast disease", "Other"),
        ("EFO_0010285", "integumentary system disease", "Other"),
        ("EFO_0001379", "endocrine system disease", "Other"),
        ("OTAR_0000010", "respiratory or thoracic disease", "Other"),
        ("EFO_0009690", "urinary system disease", "Other"),
        ("OTAR_0000006", "musculoskeletal or connective tissue disease", "Other"),
        ("MONDO_0021205", "disease of ear", "Other"),
        ("EFO_0000540", "immune system disease", "Other"),
        ("EFO_0005803", "hematologic disease", "Other"),
        ("EFO_0000618", "nervous system disease", "Other"),
        ("MONDO_0002025", "psychiatric disorder", "Other"),
        ("MONDO_0024297", "nutritional or metabolic disease", "Other"),
        ("OTAR_0000018", "genetic, familial or congenital disease", "Other"),
        ("OTAR_0000009", "injury, poisoning or other complication", "Other"),
        ("EFO_0000651", "phenotype", "Other"),
        ("EFO_0001444", "measurement", "Other"),
        ("GO_0008150", "biological process", "Other"),
    ],
    schema=StructType(
        [
            StructField("taId", StringType(), True),
            StructField("taLabel", StringType(), True),
            StructField("taLabelSimple", StringType(), True),
        ]
    ),
).withColumn("taRank", F.monotonically_increasing_id())

### give us a classification of Oncology VS non oncology
wByDisease = Window.partitionBy("diseaseId")  #### checked 31.05.2023
diseaseTA = (
    diseases.withColumn("taId", F.explode("therapeuticAreas"))
    .select(F.col("id").alias("diseaseId"), "taId", "parents")
    .join(taDf, on="taId", how="left")
    .withColumn("minRank", F.min("taRank").over(wByDisease))
    .filter(F.col("taRank") == F.col("minRank"))
    .drop("taRank", "minRank")
)

#### give us propagation of diseases and list of therapeutic areas associated
diseases2 = diseases.select("id", "parents").withColumn(
    "diseaseIdPropagated",
    F.explode_outer(F.concat(F.array(F.col("id")), F.col("parents"))),
)

chembl_trials = (
    assessment.filter((F.col("datasourceId").isin(["chembl"])))
    .groupBy("targetId", "diseaseId")
    .agg(F.max(F.col("clinicalPhase")).alias("maxClinPhase"))
)

negativeTD = (
    evidences.filter(F.col("datasourceId") == "chembl")
    .select("targetId", "diseaseId", "studyStopReason", "studyStopReasonCategories")
    .filter(F.array_contains(F.col("studyStopReasonCategories"), "Negative"))
    .groupBy("targetId", "diseaseId")
    .count()
    .withColumn("stopReason", F.lit("Negative"))
    .drop("count")
)

assessment_all = assessment.unionByName(
    gwasCredibleAssoc.withColumn("datasourceId", F.lit("gwas_credible_set")),
    allowMissingColumns=True,
)

######## 
 #evidences_all = spark.read.parquet(
 #    f"gs://open-targets-data-releases/{platform_v}/output/etl/parquet/evidence"
 #)
###
replacement_dict = {
    "gene_burden": "GeneBurden",
    "chembl": "ChEMBL",
    "intogen": "Intogen",
    "orphanet": "Orphanet",
    "cancer_gene_census": "CancerGeneCensus",
    "eva": "EvaGermline",
    "gene2phenotype": "Gene2Phenotype",
    "eva_somatic": "EvaSomatic",
    "gwas_credible_set": "OtGenetics",
    "impc": "IMPC",
}

### take only the ones with datasources for DoE
evidences_all = assessment_all.filter(F.col("datasourceId").isin(doe_sources)).replace(
    replacement_dict, subset=["datasourceId"]
)


def coincidence_matrix(evidences_all, replacement_dict):
    """Build a coincidence matrix of target disease associations between the datasources
    (these datasources are defined previously in the "evidences file passed to this function)

    Arguments:
        evidences {spark DataFrame} -- spark dataframe
        but_keep_these {list} -- list of columns to keep without checking for nulls

    Returns:
       Three spark DataFrame.show(), content: intradatasource disparities, interdatasource disparities
       and interdatasource coherency matrix coincidence matrix  -- dataframe.show()
    """

    terms = ["noEvaluable", "bivalent_risk", "null", "dispar"]
    columns = ["GoF_risk", "LoF_protect", "LoF_risk", "GoF_protect"]

    dataset1 = (  #### unique identifier and global coherency
        discrepancifier(
            # directionOfEffect(evidences, platform_v)
            evidences_all
            .withColumn("datasourceAll", F.lit("All"))
            .withColumn("niceName", F.col("datasourceId"))
            .replace(replacement_dict, subset=["niceName"])
            .filter((F.col("homogenized")).isin(terms) == False)
            .groupBy("targetId", "diseaseId")
            .pivot("homogenized")
            .agg(F.count("targetId"))
        )
        .withColumn("id", F.monotonically_increasing_id())
        .withColumnRenamed("coherencyDiagonal", "coherency_inter")
        .withColumnRenamed("coherencyOneCell", "coherency_onecell")
    ).persist()

    ### coherency intra datasource
    dataset2 = (
        discrepancifier(
            #directionOfEffect(evidences, platform_v)
            evidences_all
            .withColumn("datasourceAll", F.lit("All"))
            .withColumn("niceName", F.col("datasourceId"))
            .replace(replacement_dict, subset=["niceName"])
            .filter((F.col("homogenized")).isin(terms) == False)
            .groupBy("targetId", "diseaseId", "datasourceId")
            .pivot("homogenized")
            .agg(F.count("targetId"))
        )
        .withColumnRenamed("coherencyDiagonal", "coherency_intra")
        .withColumnRenamed("coherencyOneCell", "coherency_intra_OneCell")
    ).persist()

    ### two diferent dataset 3 depending on coherency type
    dataset3 = (
        dataset2.filter(F.col("coherency_intra") == "coherent")
        .join(dataset1, on=["targetId", "diseaseId"], how="left")
        .drop(*columns, "count")
    ).persist()

    #### cross all rows between them:
    dataset4 = (
        dataset2.join(dataset1, on=["targetId", "diseaseId"], how="left").drop(
            *columns, "count"
        )
    ).persist()

    datasetAllShared = (
        dataset2.drop("noEvaluable")
        .join(dataset1.drop("noEvaluable"), on=["targetId", "diseaseId"], how="left")
        .drop(*columns, "count")
    ).persist()

    #### matrix of Total T-D shared between datasources
    allSharedTD = (
        datasetAllShared.groupBy("datasourceId")
        .agg(F.collect_set(F.col("id")).alias("ids"))
        .selectExpr("datasourceId as datasourceId_x", "ids as ids_x")
        .join(
            datasetAllShared.groupBy("datasourceId")
            .agg(F.collect_set(F.col("id")).alias("ids"))
            .selectExpr("datasourceId as datasourceId_y", "ids as ids_y")
        )
        .withColumn(
            "sharedTD",
            F.size(F.array_intersect(F.col("ids_x"), F.col("ids_y"))),
        )
        .groupBy("datasourceId_x")
        .pivot("datasourceId_y")
        .agg(F.first(F.col("sharedTD")))
    ).persist()

    #### matrix of intradatasource disparities:
    matrix_intraDispar = (
        dataset4.groupBy("datasourceId")
        .pivot("coherency_intra")
        .agg(F.collect_set(F.col("id")))
        .selectExpr(
            "datasourceId as datasourceId_x",
            "coherent as coherent_intra",
            "dispar as dispar_intra",
        )
        .join(
            dataset4.groupBy("datasourceId")
            .pivot("coherency_intra")
            .agg(F.collect_set(F.col("id")))
            .selectExpr(
                "datasourceId as datasourceId_y",
                "coherent as coherent_intra_y",
                "dispar as dispar_intra_y",
            )
        )
        .withColumn(
            "nrs_disparIntra",
            F.size(F.array_intersect(F.col("dispar_intra"), F.col("dispar_intra_y"))),
        )
        .groupBy("datasourceId_x")
        .pivot("datasourceId_y")
        .agg(F.first(F.col("nrs_disparIntra")))
    )

    ### matrix of interdatasource disparities (including intra)

    matrix_interDispar = (
        dataset4.groupBy("datasourceId")
        .pivot("coherency_inter")
        .agg(F.collect_set(F.col("id")))
        .selectExpr(
            "datasourceId as datasourceId_x",
            "coherent as coherent_inter",
            "dispar as dispar_inter",
        )
        .join(
            dataset4.groupBy("datasourceId")
            .pivot("coherency_inter")
            .agg(F.collect_set(F.col("id")))
            .selectExpr(
                "datasourceId as datasourceId_y",
                "coherent as coherent_inter_y",
                "dispar as dispar_inter_y",
            )
        )
        .withColumn(
            "nrs_disparInter",
            F.size(F.array_intersect(F.col("dispar_inter"), F.col("dispar_inter_y"))),
        )
        .withColumn(
            "nrs_coherentInter",
            F.size(
                F.array_intersect(F.col("coherent_inter"), F.col("coherent_inter_y"))
            ),
        )
        .groupBy("datasourceId_x")
        .pivot("datasourceId_y")
        .agg(F.first(F.col("nrs_disparInter")))  #### change depending on interest
    )

    ### matrix of coincidences of coherencies

    dataset_matrix_interCoherent = (
        dataset4.groupBy("datasourceId")
        .pivot("coherency_inter")
        .agg(F.collect_set(F.col("id")))
        .selectExpr(
            "datasourceId as datasourceId_x",
            "coherent as coherent_inter",
            "dispar as dispar_inter",
        )
        .join(
            dataset4.groupBy("datasourceId")
            .pivot("coherency_inter")
            .agg(F.collect_set(F.col("id")))
            .selectExpr(
                "datasourceId as datasourceId_y",
                "coherent as coherent_inter_y",
                "dispar as dispar_inter_y",
            )
        )
        .withColumn(
            "nrs_disparInter",
            F.size(F.array_intersect(F.col("dispar_inter"), F.col("dispar_inter_y"))),
        )
        .withColumn(
            "nrs_coherentInter",
            F.size(
                F.array_intersect(F.col("coherent_inter"), F.col("coherent_inter_y"))
            ),
        )
        .withColumn(
            "nrs_Total", (F.col("nrs_disparInter") + F.col("nrs_coherentInter"))
        )
        .withColumn(
            "interCoherencyPercentage",
            (F.col("nrs_coherentInter") / F.col("nrs_Total") * 100),
        )
    )

    matrix_interCoherent = (
        dataset_matrix_interCoherent.groupBy("datasourceId_x")
        .pivot("datasourceId_y")
        .agg(F.first(F.col("nrs_coherentInter")))  #### change depending on interest
    )
    matrix_interCoherentPercentage = (
        dataset_matrix_interCoherent.groupBy("datasourceId_x")
        .pivot("datasourceId_y")
        .agg(
            F.first(F.col("interCoherencyPercentage"))
        )  #### change depending on interest
    )

    #### prepare dataset for one Cell coherencies
    dataset3OneCell = (
        dataset2.filter(F.col("coherency_intra_OneCell") == "coherent")
        .join(dataset1, on=["targetId", "diseaseId"], how="left")
        .drop(*columns, "count")
    ).persist()

    #### matrix of coherencies using oneCell criteria including only intracoherent data
    dataset_matrix_oneCellCoherent_onlyIntraCoherent = (
        dataset3OneCell.groupBy("datasourceId")
        .pivot("coherency_onecell")
        .agg(F.collect_set(F.col("id")))
        .selectExpr(
            "datasourceId as datasourceId_x",
            "coherent as coherentOneCell",
            "dispar as disparOneCell",
        )
        .join(
            dataset3OneCell.groupBy("datasourceId")
            .pivot("coherency_onecell")
            .agg(F.collect_set(F.col("id")))
            .selectExpr(
                "datasourceId as datasourceId_y",
                "coherent as coherentOneCell_y",
                "dispar as disparOneCell_y",
            )
        )
        .withColumn(
            "nrs_disparInter",
            F.size(F.array_intersect(F.col("disparOneCell"), F.col("disparOneCell_y"))),
        )
        .withColumn(
            "nrs_coherentInter",
            F.size(
                F.array_intersect(F.col("coherentOneCell"), F.col("coherentOneCell_y"))
            ),
        )
        .withColumn(
            "nrs_Total", (F.col("nrs_disparInter") + F.col("nrs_coherentInter"))
        )
        .withColumn(
            "percentageCoherencyInter",
            (F.col("nrs_coherentInter") / F.col("nrs_Total") * 100),
        )
    ).persist()

    matrix_oneCellCoherent_onlyIntraCoherent = (
        dataset_matrix_oneCellCoherent_onlyIntraCoherent.groupBy("datasourceId_x")
        .pivot("datasourceId_y")
        .agg(F.first(F.col("nrs_coherentInter")))  #### change depending on interest
    ).persist()

    matrix_oneCellCoherent_onlyIntraCoherentPercentage = (
        dataset_matrix_oneCellCoherent_onlyIntraCoherent.groupBy("datasourceId_x")
        .pivot("datasourceId_y")
        .agg(
            F.first(F.col("percentageCoherencyInter"))
        )  #### change depending on interest
    ).persist()

    ### the next are matrix combining coherency criteria and intradatasource disparities:
    matrix_interDispar_onlyIntraCoherentOneCell = (
        dataset3OneCell.groupBy("datasourceId")
        .pivot("coherency_inter")
        .agg(F.collect_set(F.col("id")))
        .selectExpr(
            "datasourceId as datasourceId_x",
            "coherent as coherent_inter",
            "dispar as dispar_inter",
        )
        .join(
            dataset3OneCell.groupBy("datasourceId")
            .pivot("coherency_inter")
            .agg(F.collect_set(F.col("id")))
            .selectExpr(
                "datasourceId as datasourceId_y",
                "coherent as coherent_inter_y",
                "dispar as dispar_inter_y",
            )
        )
        .withColumn(
            "nrs_disparInter",
            F.size(F.array_intersect(F.col("dispar_inter"), F.col("dispar_inter_y"))),
        )
        .withColumn(
            "nrs_coherentInter",
            F.size(
                F.array_intersect(F.col("coherent_inter"), F.col("coherent_inter_y"))
            ),
        )
        .groupBy("datasourceId_x")
        .pivot("datasourceId_y")
        .agg(F.first(F.col("nrs_disparInter")))  #### change depending on interest
    )

    matrix_interDispar_onlyIntraCoherent = (
        dataset3.groupBy("datasourceId")
        .pivot("coherency_inter")
        .agg(F.collect_set(F.col("id")))
        .selectExpr(
            "datasourceId as datasourceId_x",
            "coherent as coherent_inter",
            "dispar as dispar_inter",
        )
        .join(
            dataset3.groupBy("datasourceId")
            .pivot("coherency_inter")
            .agg(F.collect_set(F.col("id")))
            .selectExpr(
                "datasourceId as datasourceId_y",
                "coherent as coherent_inter_y",
                "dispar as dispar_inter_y",
            )
        )
        .withColumn(
            "nrs_disparInter",
            F.size(F.array_intersect(F.col("dispar_inter"), F.col("dispar_inter_y"))),
        )
        .withColumn(
            "nrs_coherentInter",
            F.size(
                F.array_intersect(F.col("coherent_inter"), F.col("coherent_inter_y"))
            ),
        )
        .groupBy("datasourceId_x")
        .pivot("datasourceId_y")
        .agg(F.first(F.col("nrs_disparInter")))  #### change depending on interest
    )

    matrix_interCoherent_onlyIntraCoherent = (
        dataset3.groupBy("datasourceId")
        .pivot("coherency_inter")
        .agg(F.collect_set(F.col("id")))
        .selectExpr(
            "datasourceId as datasourceId_x",
            "coherent as coherent_inter",
            "dispar as dispar_inter",
        )
        .join(
            dataset3.groupBy("datasourceId")
            .pivot("coherency_inter")
            .agg(F.collect_set(F.col("id")))
            .selectExpr(
                "datasourceId as datasourceId_y",
                "coherent as coherent_inter_y",
                "dispar as dispar_inter_y",
            )
        )
        .withColumn(
            "nrs_disparInter",
            F.size(F.array_intersect(F.col("dispar_inter"), F.col("dispar_inter_y"))),
        )
        .withColumn(
            "nrs_coherentInter",
            F.size(
                F.array_intersect(F.col("coherent_inter"), F.col("coherent_inter_y"))
            ),
        )
        .withColumn(
            "nrs_Total", (F.col("nrs_disparInter") + F.col("nrs_coherentInter"))
        )
        .withColumn(
            "percentageCoherencyInter",
            (F.col("nrs_coherentInter") / F.col("nrs_Total") * 100),
        )
        .groupBy("datasourceId_x")
        .pivot("datasourceId_y")
        .agg(
            F.first(F.col("percentageCoherencyInter"))
        )  #### change depending on interest
    )

    matrix_total_onlyIntraCoherent = (
        dataset3.groupBy("datasourceId")
        .pivot("coherency_inter")
        .agg(F.collect_set(F.col("id")))
        .selectExpr(
            "datasourceId as datasourceId_x",
            "coherent as coherent_inter",
            "dispar as dispar_inter",
        )
        .join(
            dataset3.groupBy("datasourceId")
            .pivot("coherency_inter")
            .agg(F.collect_set(F.col("id")))
            .selectExpr(
                "datasourceId as datasourceId_y",
                "coherent as coherent_inter_y",
                "dispar as dispar_inter_y",
            )
        )
        .withColumn(
            "nrs_disparInter",
            F.size(F.array_intersect(F.col("dispar_inter"), F.col("dispar_inter_y"))),
        )
        .withColumn(
            "nrs_coherentInter",
            F.size(
                F.array_intersect(F.col("coherent_inter"), F.col("coherent_inter_y"))
            ),
        )
        .withColumn(
            "nrs_Total", (F.col("nrs_disparInter") + F.col("nrs_coherentInter"))
        )
        .withColumn(
            "percentageCoherencyInter",
            (F.col("nrs_coherentInter") / F.col("nrs_Total") * 100),
        )
        .groupBy("datasourceId_x")
        .pivot("datasourceId_y")
        .agg(F.first(F.col("nrs_Total")))  #### change depending on interest
    )

    return (
        allSharedTD,  ### N T-D shared between all DS
        matrix_intraDispar,  ### intradatasource disparities
        matrix_interDispar,  ### using diagonal criteria, from coherent TD per DS, dispar TD shared between DS
        matrix_interCoherent,  ### using diagonal criteria, from coherent TD per DS, coherent TD shared between DS
        matrix_interCoherentPercentage,  ### same as above but % of coherent across all TD shared when being coherent intra DS
        matrix_oneCellCoherent_onlyIntraCoherent,  ### using one cell criteria, from coherent TD per DS, coherent TD shared between DS
        matrix_oneCellCoherent_onlyIntraCoherentPercentage,  ### same as above but % of coherent TD over dispar between DS, when being coherent intra DS
        matrix_interDispar_onlyIntraCoherentOneCell,  ### using one cell criteria to check intra coherency, N of TD dispar shared between DS
        matrix_interDispar_onlyIntraCoherent,  #### using diagonal criteria to check intra coherency, N of TD dispar shared between DS
        matrix_interCoherent_onlyIntraCoherent,  #### using diagonal criteria, percentage of shared T-D between datasources that are coherent
        matrix_total_onlyIntraCoherent,  ### using diagonal criteria, sum of shared TD dispar and coherent between DS
    )

loaded files
loaded newColoc


loaded gwasComplete
loaded resolvedColloc


25/05/08 06:09:58 WARN CacheManager: Asked to cache already cached data.


run temporary direction of effect
Moving to step 2


25/05/08 06:09:59 WARN CacheManager: Asked to cache already cached data.


In [ ]:
listado = coincidence_matrix(evidences_all, replacement_dict)

ERROR:root:KeyboardInterrupt while sending command.             (82 + 63) / 207]]7]
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/miniconda3/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
len(listado)

In [ ]:
(
    allSharedTD,
    matrix_intraDispar,
    matrix_interDispar,
    matrix_interCoherent,
    matrix_interCoherentPercentage,
    matrix_oneCellCoherent_onlyIntraCoherent,
    matrix_oneCellCoherent_onlyIntraCoherentPercentage,
    matrix_interDispar_onlyIntraCoherentOneCell,
    matrix_interDispar_onlyIntraCoherent,
    matrix_interCoherent_onlyIntraCoherent,
    matrix_total_onlyIntraCoherent
) = coincidence_matrix(evidences_all, replacement_dict)

25/05/08 06:12:35 WARN CacheManager: Asked to cache already cached data.         46]
25/05/08 06:12:35 WARN CacheManager: Asked to cache already cached data.


In [ ]:
allSharedTD.toPandas().to_csv('gs://ot-team/jroldan/analysis/discrepancies/allsahredTD.csv')

In [11]:
evidences_all.filter(F.col('datasourceId')=='OtGenetics').groupBy('homogenized').count().show()

+-----------+-------+
|homogenized|  count|
+-----------+-------+
|GoF_protect|3858394|
|   GoF_risk|4013521|
|LoF_protect|3609955|
|   LoF_risk|3651326|
+-----------+-------+



In [6]:
allSharedTD.show()

+----------------+----------------+------+-----------+----------+--------------+----------+------+-------+--------+----------+
|  datasourceId_x|CancerGeneCensus|ChEMBL|EvaGermline|EvaSomatic|Gene2Phenotype|GeneBurden|  IMPC|Intogen|Orphanet|OtGenetics|
+----------------+----------------+------+-----------+----------+--------------+----------+------+-------+--------+----------+
|      OtGenetics|              46|   314|        105|         1|             1|      2178|   210|      4|       1|    222011|
|CancerGeneCensus|           35064|  1439|        109|        62|             5|        85|   330|    726|       2|        46|
|         Intogen|             726|   198|         22|        23|             1|        16|    52|   2349|       0|         4|
|  Gene2Phenotype|               5|     3|       1451|        27|          1972|         5|   378|      1|     250|         1|
|        Orphanet|               2|    11|        719|         8|           250|         0|   297|      0|    1

In [7]:
matrix_intraDispar.show()

+----------------+----------------+------+-----------+----------+--------------+----------+----+-------+--------+----------+
|  datasourceId_x|CancerGeneCensus|ChEMBL|EvaGermline|EvaSomatic|Gene2Phenotype|GeneBurden|IMPC|Intogen|Orphanet|OtGenetics|
+----------------+----------------+------+-----------+----------+--------------+----------+----+-------+--------+----------+
|      OtGenetics|               0|     1|          0|         0|             0|        29|   0|      0|       0|     40282|
|CancerGeneCensus|               0|     0|          0|         0|             0|         0|   0|      0|       0|         0|
|  Gene2Phenotype|               0|     0|          0|         0|             0|         0|   0|      0|       0|         0|
|        Orphanet|               0|     0|          0|         0|             0|         0|   0|      0|       1|         0|
|            IMPC|               0|     0|          0|         0|             0|         0|   0|      0|       0|         0|


In [9]:
matrix_interCoherent.show()

+----------------+----------------+------+-----------+----------+--------------+----------+------+-------+--------+----------+
|  datasourceId_x|CancerGeneCensus|ChEMBL|EvaGermline|EvaSomatic|Gene2Phenotype|GeneBurden|  IMPC|Intogen|Orphanet|OtGenetics|
+----------------+----------------+------+-----------+----------+--------------+----------+------+-------+--------+----------+
|      OtGenetics|              14|   140|         41|         0|             0|      1149|    76|      1|       0|    181034|
|CancerGeneCensus|           34506|  1156|         83|        53|             5|        53|   231|    545|       1|        14|
|         Intogen|             545|   135|         18|        16|             1|        10|    34|   2120|       0|         1|
|  Gene2Phenotype|               5|     3|       1445|        27|          1962|         4|   374|      1|     244|         0|
|        Orphanet|               1|     3|        663|         7|           244|         0|   246|      0|    1

In [8]:
matrix_interDispar.show()

+----------------+----------------+------+-----------+----------+--------------+----------+----+-------+--------+----------+
|  datasourceId_x|CancerGeneCensus|ChEMBL|EvaGermline|EvaSomatic|Gene2Phenotype|GeneBurden|IMPC|Intogen|Orphanet|OtGenetics|
+----------------+----------------+------+-----------+----------+--------------+----------+----+-------+--------+----------+
|      OtGenetics|              32|   174|         64|         1|             1|      1029| 134|      3|       1|     40977|
|CancerGeneCensus|             558|   283|         26|         9|             0|        32|  99|    181|       1|        32|
|         Intogen|             181|    63|          4|         7|             0|         6|  18|    229|       0|         3|
|  Gene2Phenotype|               0|     0|          6|         0|            10|         1|   4|      0|       6|         1|
|        Orphanet|               1|     8|         56|         1|             6|         0|  51|      0|      93|         1|


In [ ]:
#########
#### test the matrices for discrepancies  #### 
#########

In [3]:
evidences_all.groupBy('datasourceId').count().show()

+----------------+--------+
|    datasourceId|   count|
+----------------+--------+
|CancerGeneCensus|   82754|
|         Intogen|    4224|
|  Gene2Phenotype|    4181|
|        Orphanet|    6293|
|            IMPC| 1175688|
|      GeneBurden|   36805|
|      EvaSomatic|    9719|
|     EvaGermline| 3138383|
|          ChEMBL|  573124|
|      OtGenetics|15133196|
+----------------+--------+



In [ ]:
### substitute all old coloc by new one

In [1]:
#### BUILDING THE NEW GWAS GENETIC EVIDENCE FROM COLOC
from functions import (
    discrepancifier,
    build_gwasResolvedColoc,
    temporary_directionOfEffect,
    spreadSheetFormatter,
)
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F
import matplotlib.pyplot as plt
from datetime import date, datetime
from pyspark.sql.types import (
    StructType,
    StructField,
    ArrayType,
    DoubleType,
    DecimalType,
    StringType,
    FloatType,
    IntegerType,
)
import pandas as pd

spark = SparkSession.builder.getOrCreate()

# path = "gs://open-targets-pre-data-releases/24.12-uo_test-3/output/etl/parquet/"

path_n='gs://open-targets-data-releases/25.03/output/'


evidences = (
    spark.read.parquet(f"{path_n}evidence").filter(
        F.col("datasourceId").isin(
            [
                "gwas_credible_sets",
                "chembl",
            ]
        )
    )
    # .persist()
)

negativeTD = (
    evidences.filter(F.col("datasourceId") == "chembl")
    .select("targetId", "diseaseId", "studyStopReason", "studyStopReasonCategories")
    .filter(F.array_contains(F.col("studyStopReasonCategories"), "Negative"))
    .groupBy("targetId", "diseaseId")
    .count()
    .withColumn("stopReason", F.lit("Negative"))
    .drop("count")
)

#### include here right way of generating gwasResolvedColoc
#### resolved coloc: 
diseases = spark.read.parquet(f"{path_n}disease/")

credible = spark.read.parquet(f"{path_n}credible_set")

new = spark.read.parquet(f"{path_n}colocalisation_coloc") 

index=spark.read.parquet(f"{path_n}study/")

variantIndex = spark.read.parquet(f"{path_n}variant")

biosample = spark.read.parquet(f"{path_n}biosample")

print("loaded files")

newColoc = (
    new.join(
        credible.selectExpr(  #### studyLocusId from credible set to uncover the codified variants on left side
            "studyLocusId as leftStudyLocusId",
            "StudyId as leftStudyId",
            "variantId as leftVariantId",
            "studyType as credibleLeftStudyType",
        ),
        on="leftStudyLocusId",
        how="left",
    )
    .join(
        credible.selectExpr(  #### studyLocusId from credible set to uncover the codified variants on right side
            "studyLocusId as rightStudyLocusId",
            "studyId as rightStudyId",
            "variantId as rightVariantId",
            "studyType as credibleRightStudyType",
            'isTransQtl'
        ),
        on="rightStudyLocusId",
        how="left",
    )
    .join(
        index.selectExpr(  ### bring modulated target on right side (QTL study)
            "studyId as rightStudyId",
            "geneId",
            "projectId",
            "studyType as indexStudyType",
            "condition",
            "biosampleId",
        ),
        on="rightStudyId",
        how="left",
)
    # .persist()
)

print("loaded newColoc")

# remove columns without content (only null values on them)
df = evidences.filter((F.col("datasourceId") == "gwas_credible_sets"))

# Use an aggregation to determine non-null columns
non_null_counts = df.select(
    *[F.sum(F.col(col).isNotNull().cast("int")).alias(col) for col in df.columns]
)

# Collect the counts for each column
non_null_columns = [
    row[0] for row in non_null_counts.collect()[0].asDict().items() if row[1] > 0
]

# Select only the non-null columns
filtered_df = df.select(*non_null_columns)  # .persist()

## bring studyId, variantId, beta from Gwas and pValue
gwasComplete = filtered_df.join(
    credible.selectExpr(
        "studyLocusId", "studyId", "variantId", "beta as betaGwas", "pValueExponent"
    ),
    on="studyLocusId",
    how="left",
)  # .persist()

print("loaded gwasComplete")

resolvedColoc = (
    (
        newColoc.withColumnRenamed("geneId", "targetId")
        .join(
            gwasComplete.withColumnRenamed("studyLocusId", "leftStudyLocusId"),
            on=["leftStudyLocusId", "targetId"],
            how="inner",
        )
        .join(  ### propagated using parent terms
            diseases.selectExpr(
                "id as diseaseId", "name", "parents", "therapeuticAreas"
            ),
            on="diseaseId",
            how="left",
        )
        .withColumn(
            "diseaseId",
            F.explode_outer(F.concat(F.array(F.col("diseaseId")), F.col("parents"))),
        )
        .drop("parents", "oldDiseaseId")
    ).withColumn(
        "colocDoE",
        F.when(
            F.col("rightStudyType").isin(
                ["eqtl", "pqtl", "tuqtl", "sceqtl", "sctuqtl"]
            ),
            F.when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("GoF_risk"),
            )
            .when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("LoF_risk"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("LoF_protect"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("GoF_protect"),
            ),
        ).when(
            F.col("rightStudyType").isin(
                ["sqtl", "scsqtl"]
            ),  ### opposite directionality than sqtl
            F.when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("LoF_risk"),
            )
            .when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("GoF_risk"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("GoF_protect"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("LoF_protect"),
            ),
        ),
    )
    # .persist()
)
print("loaded resolvedColloc")

##### 


### alternative for gwas credible assoc 
window_spec = Window.partitionBy("targetId", "diseaseId",'leftStudyId').orderBy( ### include gwas study
    F.col("pValueExponent").asc()
)
gwasCredibleAssoc = (
    resolvedColoc.withColumn(
        "homogenized", F.first("colocDoE", ignorenulls=True).over(window_spec)
    )  ## added 30.01.2025
    .select(
        "targetId",
        "diseaseId",
        "homogenized",
        "leftStudyLocusId",
        "h4",
        "datasourceId",
        "resourceScore",
        "leftVariantId",
        "credibleLeftStudyType",
    )
    .withColumn(
        "homogenized",
        F.when(F.col("homogenized").isNull(), F.lit("noEvaluable")).otherwise(
            F.col("homogenized")
        ),
    )
)


#### LOAD STUDYLOCUSID AND VARIANT DISTANCES

l2gPred=spark.read.parquet(f"{path_n}l2g_prediction/")

l2gTable = (
    l2gPred.select("studyLocusId", "geneId", F.explode_outer("features")).select('studyLocusId','geneId','col.name','col.value').filter(
        F.col("col.name").isin(["distanceFootprintMean", "distanceTssMean"])
    )
    .groupBy("studyLocusId", "geneId")
    .pivot("name")
    .agg(F.first("value"))
)

print("creating gwasCredibleAssocDistances")
gwasCredibleAssocDistances = gwasCredibleAssoc.join(
    l2gTable.withColumnRenamed("studyLocusId", "leftStudyLocusId").withColumnRenamed(
        "geneId", "targetId"
    ),
    on=["leftStudyLocusId", "targetId"],
    how="left",
)

print("creating analysis_chembl")

datasource_filter = ["chembl"]
preanalysis_chembl, evidences, actionType, oncolabel = temporary_directionOfEffect(
    path_n, datasource_filter
)
analysis_chembl = (
    discrepancifier(
        preanalysis_chembl.withColumn(
            "maxClinPhase",
            F.max(F.col("clinicalPhase")).over(
                Window.partitionBy("targetId", "diseaseId")
            ),
        )
        .groupBy("targetId", "diseaseId", "maxClinPhase")
        .pivot("homogenized")
        .agg(F.count("targetId"))
        .persist()
    )  #### avoid the
    #    .filter(  ### ensure drug has annotated MoA and is coherent per Target-Disease
    #        ((F.col("GoF_protect").isNotNull()) | F.col("LoF_protect").isNotNull())
    #        & (F.col("coherencyDiagonal") == "coherent")
).selectExpr(
    "targetId",
    "diseaseId",
    "maxClinPhase",
    "coherencyDiagonal as coherencyDiagonal_ch",
    "coherencyOneCell as coherencyOneCell_ch",
    "LoF_protect as LoF_protect_ch",
    "GoF_protect as GoF_protect_ch",
)

### NEWWWWW ### 
doe_columns=["LoF_protect", "GoF_risk", "LoF_risk", "GoF_protect"]
diagonal_lof=['LoF_protect','GoF_risk']
diagonal_gof=['LoF_risk','GoF_protect']

diseases2 = diseases.select("id", "parents").withColumn(
    "diseaseIdPropagated",
    F.explode_outer(F.concat(F.array(F.col("id")), F.col("parents"))),
)
### pivot colocdoE grouping by T-D-studyLocusId-distances

value = "max_L2GScore"

list_l2g = [
    0.10,
    0.15,
    0.20,
    0.25,
    0.30,
    0.35,
    0.40,
    0.45,
    0.50,
    0.55,
    0.60,
    0.65,
    0.70,
    0.72,
    0.74,
    0.75,
    0.76,
    0.78,
    0.80,
    0.82,
    0.84,
    0.85,
    0.86,
    0.88,
    0.90,
    0.92,
    0.94,
    0.95,
    0.96,
    0.98,
]
print("creating benchmarkOT function")

dict_comb = {}


def benchmarkOT(
    dict_comb,
    value,
    gwasCredibleAssocDistances,
    analysis_chembl,
    negativeTD,
    list_l2g,
):
    dict_comb = {
        "hasGeneticEvidence": f"{value}",
        "diagonalYes": f"{value}",
        "oneCellYes": f"{value}",
        "L2GAndColoc": f"{value}",
        'NoneCellYes': f"{value}",
        'NdiagonalYes': f"{value}",
    }
    min_value = gwasCredibleAssocDistances.agg(
        F.min("distanceFootprintMean")
    ).collect()[0][
        0
    ]  ## take min value for scalating
    conditions = [
        F.when(F.col(c) == F.col("maxDoE"), F.lit(c)).otherwise(F.lit(None)) for c in doe_columns
        ]

    dataset_original = discrepancifier(
        gwasCredibleAssocDistances
        # .filter(F.col("h4").isNotNull()) #### not filter by this because we want to include the L2G AND Coloc question
        .withColumn(  ### take maximum L2G score per T-D
            "max_L2GScore",
            F.max("resourceScore").over(Window.partitionBy("targetId", "diseaseId")),
        )
        .withColumn("min_distanceFootprintMean", F.lit(min_value))
        .withColumn(
            "scaled_distanceFootprintMean",
            (F.col("distanceFootprintMean") - F.col("min_distanceFootprintMean"))
            / (1 - F.col("min_distanceFootprintMean")),
        )
        .withColumn(
            "minDistFootPrintMean",
            F.min("scaled_distanceFootprintMean").over(
                Window.partitionBy("targetId", "diseaseId")
            ),
        )
        .withColumn(
            "minTssDistance",
            F.min("distanceTssMean").over(Window.partitionBy("targetId", "diseaseId")),
        )
        .groupBy(
            "targetId",
            "diseaseId",
            f"{value}",
            # "leftStudyLocusId",
        )
        .pivot("homogenized")
        .count()
    )
    dataset_propag = discrepancifier(
        gwasCredibleAssocDistances
        # .filter(F.col("h4").isNotNull()) #### not filter by this because we want to include the L2G AND Coloc question
        .withColumn(  ### take maximum L2G score per T-D
            "max_L2GScore",
            F.max("resourceScore").over(Window.partitionBy("targetId", "diseaseId")),
        )
        .withColumn("min_distanceFootprintMean", F.lit(min_value))
        .withColumn(
            "scaled_distanceFootprintMean",
            (F.col("distanceFootprintMean") - F.col("min_distanceFootprintMean"))
            / (1 - F.col("min_distanceFootprintMean")),
        )
        .withColumn(
            "minDistFootPrintMean",
            F.min("scaled_distanceFootprintMean").over(
                Window.partitionBy("targetId", "diseaseId")
            ),
        )
        .withColumn(
            "minTssDistance",
            F.min("distanceTssMean").over(Window.partitionBy("targetId", "diseaseId")),
        )
        .join(
            diseases2.selectExpr("id as diseaseId", "diseaseIdPropagated"),
            on="diseaseId",
            how="left",
        )
        .withColumnRenamed("diseaseId", "oldDiseaseId")
        .withColumnRenamed("diseaseIdPropagated", "diseaseId")
        .groupBy(
            "targetId",
            "diseaseId",
            f"{value}",
            # "leftStudyLocusId",
        )
        .pivot("homogenized")
        .count()
    )

    return (
        dataset_original.join(
            analysis_chembl, on=["targetId", "diseaseId"], how="right"
        )
        .withColumn(
            "diagonalAgreeWithDrugs",
            F.when(
                (
                    (F.col("coherencyDiagonal_ch") == "coherent")
                    & (F.col("coherencyDiagonal") == "coherent")
                )
                # & (F.col("coherencyDiagonal") == "coherent")
                ,
                F.when(
                    (F.col("LoF_protect_ch").isNotNull())
                    & (
                        F.col("GoF_risk").isNotNull() | F.col("LoF_protect").isNotNull()
                    ),
                    F.lit("coherent"),
                )
                .when(
                    F.col("GoF_protect_ch").isNotNull()
                    & (
                        F.col("LoF_risk").isNotNull() | F.col("GoF_protect").isNotNull()
                    ),
                    F.lit("coherent"),
                )
                .otherwise(F.lit("dispar")),
            ),
        )
        .withColumn(
            "oneCellAgreeWithDrugs",
            F.when(
                (
                    (F.col("coherencyOneCell_ch") == "coherent")
                    & (F.col("coherencyDiagonal") == "coherent")
                )
                # & (F.col("coherencyOneCell") == "coherent")
                ,
                F.when(
                    (F.col("LoF_protect_ch").isNotNull())
                    & (
                        (F.col("LoF_protect").isNotNull())
                        & (F.col("LoF_risk").isNull())
                        & (F.col("GoF_protect").isNull())
                        & (F.col("GoF_risk").isNull())
                    ),
                    F.lit("coherent"),
                )
                .when(
                    (F.col("GoF_protect_ch").isNotNull())
                    & (
                        (F.col("GoF_protect").isNotNull())
                        & (F.col("LoF_risk").isNull())
                        & (F.col("LoF_protect").isNull())
                        & (F.col("GoF_risk").isNull())
                    ),
                    F.lit("coherent"),
                )
                .otherwise(F.lit("dispar")),
            ),
            # ).filter(
            #    F.col("diagonalAgreeWithDrugs").isNotNull()
        ).withColumn(
            "arrayN", F.array(*[F.col(c) for c in doe_columns])
        ).withColumn(
            "maxDoE", F.array_max(F.col("arrayN"))
        ).withColumn("maxDoE_names", F.array(*conditions)
        ).withColumn("maxDoE_names", F.expr("filter(maxDoE_names, x -> x is not null)")
        )
        .withColumn(
            "Phase4",
            F.when(F.col("maxClinPhase") == 4, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "Phase>=3",
            F.when(F.col("maxClinPhase") >= 3, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "Phase>=2",
            F.when(F.col("maxClinPhase") >= 2, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "Phase>=1",
            F.when(F.col("maxClinPhase") >= 1, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "Phase0",
            F.when(F.col("maxClinPhase") == 0, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "diagonalYes",
            F.when(
                F.col("diagonalAgreeWithDrugs") == "coherent", F.lit("yes")
            ).otherwise(F.lit("no")),
        )
        .withColumn(
            "oneCellYes",
            F.when(
                F.col("oneCellAgreeWithDrugs") == "coherent", F.lit("yes")
            ).otherwise(F.lit("no")),
        )
        .withColumn(
            "hasGeneticEvidence",
            F.when(F.col(f"{value}").isNotNull(), F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "L2GAndColoc",
            F.when(
                (F.col(f"{value}").isNotNull())
                & (F.col("coherencyDiagonal").isin(["coherent", "dispar"])),
                F.lit("yes"),
            ).otherwise(F.lit("no")),
        )
        .withColumn(
            "maxDoEArrayN",
            F.expr("aggregate(arrayN, 0, (acc, x) -> acc + IF(x = maxDoE, 1, 0))")
        ).withColumn(
            "NoneCellYes",
            F.when(F.col("LoF_protect_ch").isNotNull() & (F.array_contains(F.col("maxDoE_names"), F.lit("LoF_protect")))==True, F.lit('yes'))
            .when(F.col("GoF_protect_ch").isNotNull() & (F.array_contains(F.col("maxDoE_names"), F.lit("GoF_protect")))==True, F.lit('yes')
                ).otherwise(F.lit('no'))  # If the value is null, return null # Otherwise, check if name is in array
        ).withColumn(
            "NdiagonalYes",
            F.when(F.col("LoF_protect_ch").isNotNull() & 
                (F.size(F.array_intersect(F.col("maxDoE_names"), F.array([F.lit(x) for x in diagonal_lof]))) > 0),
                F.lit("yes")
            ).when(F.col("GoF_protect_ch").isNotNull() & 
                (F.size(F.array_intersect(F.col("maxDoE_names"), F.array([F.lit(x) for x in diagonal_gof]))) > 0),
                F.lit("yes")
            ).otherwise(F.lit('no'))
        )
        .join(negativeTD, on=["targetId", "diseaseId"], how="left")
        .withColumn(
            "PhaseT",
            F.when(F.col("stopReason") == "Negative", F.lit("yes")).otherwise(F.lit("no")),
        )
        .select(
            ["*"]
            + (
                [  ### single columns
                    F.when(F.col(f"{value}") >= n, F.lit("yes"))
                    .otherwise(F.lit("no"))
                    .alias(f"{value}>={str(n).replace('.', '_')}")
                    for n in list_l2g
                ]
            )
            + (
                [  ### column combinations for Yes/No colums Plus has DoE (any agreement)
                    F.when((F.col(a) == "yes") & (F.col(x) >= n), F.lit("yes"))
                    .otherwise(F.lit("no"))
                    .alias(f"{x}>={str(n).replace('.', '_')}&{a}_combined")
                    for a, x in dict_comb.items()
                    for n in list_l2g
                ]
            )
        ),
        dataset_propag.join(
            analysis_chembl, on=["targetId", "diseaseId"], how="right"
        )
        .withColumn(
            "diagonalAgreeWithDrugs",
            F.when(
                (
                    (F.col("coherencyDiagonal_ch") == "coherent")
                    & (F.col("coherencyDiagonal") == "coherent")
                )
                # & (F.col("coherencyDiagonal") == "coherent")
                ,
                F.when(
                    (F.col("LoF_protect_ch").isNotNull())
                    & (
                        F.col("GoF_risk").isNotNull() | F.col("LoF_protect").isNotNull()
                    ),
                    F.lit("coherent"),
                )
                .when(
                    F.col("GoF_protect_ch").isNotNull()
                    & (
                        F.col("LoF_risk").isNotNull() | F.col("GoF_protect").isNotNull()
                    ),
                    F.lit("coherent"),
                )
                .otherwise(F.lit("dispar")),
            ),
        )
        .withColumn(
            "oneCellAgreeWithDrugs",
            F.when(
                (
                    (F.col("coherencyOneCell_ch") == "coherent")
                    & (F.col("coherencyDiagonal") == "coherent")
                )
                # & (F.col("coherencyOneCell") == "coherent")
                ,
                F.when(
                    (F.col("LoF_protect_ch").isNotNull())
                    & (
                        (F.col("LoF_protect").isNotNull())
                        & (F.col("LoF_risk").isNull())
                        & (F.col("GoF_protect").isNull())
                        & (F.col("GoF_risk").isNull())
                    ),
                    F.lit("coherent"),
                )
                .when(
                    (F.col("GoF_protect_ch").isNotNull())
                    & (
                        (F.col("GoF_protect").isNotNull())
                        & (F.col("LoF_risk").isNull())
                        & (F.col("LoF_protect").isNull())
                        & (F.col("GoF_risk").isNull())
                    ),
                    F.lit("coherent"),
                )
                .otherwise(F.lit("dispar")),
            ),
            # ).filter(
            #    F.col("diagonalAgreeWithDrugs").isNotNull()
        ).withColumn(
            "arrayN", F.array(*[F.col(c) for c in doe_columns])
        ).withColumn(
            "maxDoE", F.array_max(F.col("arrayN"))
        ).withColumn("maxDoE_names", F.array(*conditions)
        ).withColumn("maxDoE_names", F.expr("filter(maxDoE_names, x -> x is not null)")
        )
        .withColumn(
            "Phase4",
            F.when(F.col("maxClinPhase") == 4, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "Phase>=3",
            F.when(F.col("maxClinPhase") >= 3, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "Phase>=2",
            F.when(F.col("maxClinPhase") >= 2, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "Phase>=1",
            F.when(F.col("maxClinPhase") >= 1, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "Phase0",
            F.when(F.col("maxClinPhase") == 0, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "diagonalYes",
            F.when(
                F.col("diagonalAgreeWithDrugs") == "coherent", F.lit("yes")
            ).otherwise(F.lit("no")),
        )
        .withColumn(
            "oneCellYes",
            F.when(
                F.col("oneCellAgreeWithDrugs") == "coherent", F.lit("yes")
            ).otherwise(F.lit("no")),
        )
        .withColumn(
            "hasGeneticEvidence",
            F.when(F.col(f"{value}").isNotNull(), F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "L2GAndColoc",
            F.when(
                (F.col(f"{value}").isNotNull())
                & (F.col("coherencyDiagonal").isin(["coherent", "dispar"])),
                F.lit("yes"),
            ).otherwise(F.lit("no")),
        )
        .withColumn(
            "maxDoEArrayN",
            F.expr("aggregate(arrayN, 0, (acc, x) -> acc + IF(x = maxDoE, 1, 0))")
        ).withColumn(
            "NoneCellYes",
            F.when(F.col("LoF_protect_ch").isNotNull() & (F.array_contains(F.col("maxDoE_names"), F.lit("LoF_protect")))==True, F.lit('yes'))
            .when(F.col("GoF_protect_ch").isNotNull() & (F.array_contains(F.col("maxDoE_names"), F.lit("GoF_protect")))==True, F.lit('yes')
                ).otherwise(F.lit('no'))  # If the value is null, return null # Otherwise, check if name is in array
        ).withColumn(
            "NdiagonalYes",
            F.when(F.col("LoF_protect_ch").isNotNull() & 
                (F.size(F.array_intersect(F.col("maxDoE_names"), F.array([F.lit(x) for x in diagonal_lof]))) > 0),
                F.lit("yes")
            ).when(F.col("GoF_protect_ch").isNotNull() & 
                (F.size(F.array_intersect(F.col("maxDoE_names"), F.array([F.lit(x) for x in diagonal_gof]))) > 0),
                F.lit("yes")
            ).otherwise(F.lit('no'))
        )
        .join(negativeTD, on=["targetId", "diseaseId"], how="left")
        .withColumn(
            "PhaseT",
            F.when(F.col("stopReason") == "Negative", F.lit("yes")).otherwise(F.lit("no")),
        )
        .select(
            ["*"]
            + (
                [  ### single columns
                    F.when(F.col(f"{value}") >= n, F.lit("yes"))
                    .otherwise(F.lit("no"))
                    .alias(f"{value}>={str(n).replace('.', '_')}")
                    for n in list_l2g
                ]
            )
            + (
                [  ### column combinations for Yes/No colums Plus has DoE (any agreement)
                    F.when((F.col(a) == "yes") & (F.col(x) >= n), F.lit("yes"))
                    .otherwise(F.lit("no"))
                    .alias(f"{x}>={str(n).replace('.', '_')}&{a}_combined")
                    for a, x in dict_comb.items()
                    for n in list_l2g
                ]
            )
        )
    )

print("creating dataframes in loop")

values = [
    "max_L2GScore",
    "minDistFootPrintMean",
    "minTssDistance",
]

datasetDict = {}
datasetDict_propag = {}

for value in values:
    if value == "max_L2GScore":
        (
            datasetDict[f"max_L2GScore_original"],
            datasetDict_propag[f"max_L2GScore_propag"],
        ) = benchmarkOT(
            dict_comb,
            value,
            gwasCredibleAssocDistances,
            analysis_chembl,
            negativeTD,
            list_l2g,
        )
    else:
        datasetDict[f"{value}_original"], datasetDict_propag[f"{value}_propag"] = (
            benchmarkOT(
                dict_comb,
                value,
                gwasCredibleAssocDistances,
                analysis_chembl,
                negativeTD,
                list_l2g,
            )
        )



def comparisons_df(dataset) -> list:
    """Return list of all comparisons to be used in the analysis"""
    toAnalysis = dataset.drop("clinicalStatus", "stopReason", "PhaseT",'maxDoEArrayN').columns[25:]
    dataType = ["byDatatype"] * len(toAnalysis)
    l_studies = []
    l_studies.extend([list(a) for a in zip(toAnalysis, dataType)])

    schema = StructType(
        [
            StructField("comparison", StringType(), True),
            StructField("comparisonType", StringType(), True),
        ]
    )

    comparisons = spark.createDataFrame(l_studies, schema=schema)
    ### include all the columns as predictor

    predictions = spark.createDataFrame(
        data=[
            ("Phase4", "clinical"),
            ("Phase>=3", "clinical"),
            ("Phase>=2", "clinical"),
            ("Phase>=1", "clinical"),
            ("PhaseT", "clinical"),
        ]
    )
    return comparisons.join(predictions, how="full").collect()


from functions import relative_success, spreadSheetFormatter, convertTuple

full_data = spark.createDataFrame(
    data=[
        ("yes", "yes"),
        ("yes", "no"),
        ("no", "yes"),
        ("no", "no"),
    ],
    schema=StructType(
        [
            StructField("prediction", StringType(), True),
            StructField("comparison", StringType(), True),
        ]
    ),
)


def aggregations_original(
    df,
    data,
    listado,
    comparisonColumn,
    comparisonType,
    predictionColumn,
    predictionType,
    today_date,
):

    wComparison = Window.partitionBy(F.col(comparisonColumn))
    wPrediction = Window.partitionBy(F.col(predictionColumn))
    wPredictionComparison = Window.partitionBy(
        F.col(comparisonColumn), F.col(predictionColumn)
    )

    uniqIds = df.select("targetId", "diseaseId").distinct().count()
    out = (
        df.withColumn("comparisonType", F.lit(comparisonType))
        .withColumn("predictionType", F.lit(predictionType))
        .withColumn("total", F.lit(uniqIds))
        .withColumn("a", F.count("targetId").over(wPredictionComparison))
        .withColumn(
            "predictionTotal",
            F.count("targetId").over(wPrediction),
        )
        .withColumn(
            "comparisonTotal",
            F.count("targetId").over(wComparison),
        )
        .select(
            F.col(predictionColumn).alias("prediction"),
            F.col(comparisonColumn).alias("comparison"),
            "comparisonType",
            "predictionType",
            "a",
            "predictionTotal",
            "comparisonTotal",
            "total",
        )
        .filter(F.col("prediction").isNotNull())
        .filter(F.col("comparison").isNotNull())
        .distinct()
    )
    out.write.mode("overwrite").parquet(
        "gs://ot-team/jroldan/"
        + str(
            today_date
            + "_"
            + "analysis_L2G/"
            + data
            # + "_propagated"
            + "/"
            + comparisonColumn
            + "_"
            + predictionColumn
            + ".parquet"
        )
    )
    filePath = "gs://ot-team/jroldan/" + str(
        today_date
        + "_"
        + "analysis_L2G/"
        + data
        # + "_propagated"
        + "/"
        + comparisonColumn
        + "_"
        + predictionColumn
        + ".parquet"
    )

    listado.append(
        "gs://ot-team/jroldan/"
        + str(
            today_date
            + "_"
            + "analysis_L2G/"
            + data
            # + "_propagated"
            + "/"
            + comparisonColumn
            + "_"
            + predictionColumn
            + ".parquet"
        )
    )
    print(
        today_date
        + "_"
        + "analysis_L2G/"
        + data
        # + "_propagated"
        + "/"
        + comparisonColumn
        + "_"
        + predictionColumn
        + ".parquet"
    )

    c = datetime.now()
    c.strftime("%H:%M:%S")
    print(c)
    array1 = np.delete(
        out.join(full_data, on=["prediction", "comparison"], how="outer")
        .groupBy("comparison")
        .pivot("prediction")
        .agg(F.first("a"))
        .sort(F.col("comparison").desc())
        .select("comparison", "yes", "no")
        .fillna(0)
        .toPandas()
        .to_numpy(),
        [0],
        1,
    )
    total = np.sum(array1)
    res_npPhaseX = np.array(array1, dtype=int)
    resX = convertTuple(fisher_exact(res_npPhaseX, alternative="two-sided"))
    resx_CI = convertTuple(
        odds_ratio(res_npPhaseX).confidence_interval(confidence_level=0.95)
    )

    result_st.append(resX)
    result_ci.append(resx_CI)
    (rs_result, rs_ci) = relative_success(array1)

    results.append(
        [
            data,
            comparisonColumn,
            predictionColumn,
            round(float(resX.split(",")[0]), 2),
            float(resX.split(",")[1]),
            round(float(resx_CI.split(",")[0]), 2),
            round(float(resx_CI.split(",")[1]), 2),
            str(total),
            np.array(res_npPhaseX).tolist(),
            round(float(rs_result), 2),
            round(float(rs_ci[0]), 2),
            round(float(rs_ci[1]), 2),
            filePath,
        ]
    )
    return results


import pandas as pd
import numpy as np
from scipy.stats import fisher_exact
from scipy.stats.contingency import odds_ratio

# Initialize an empty list to store the results
result_st = []
result_ci = []
results = []


def convertTuple(tup):
    st = ",".join(map(str, tup))
    return st


print("launched function to run analysis")


listado = []
today_date = str(date.today())
aggSetups_original = comparisons_df(datasetDict["max_L2GScore_original"])



25/05/08 05:59:50 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/05/08 06:00:05 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/05/08 06:00:20 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/05/08 06:00:35 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/05/08 06:00:50 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/05/08 06:01:05 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registere

loaded files
loaded newColoc


25/05/08 06:03:31 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


loaded gwasComplete
loaded resolvedColloc


creating gwasCredibleAssocDistances
creating analysis_chembl


creating benchmarkOT function
creating dataframes in loop


launched function to run analysis


In [4]:
datasetDict["max_L2GScore_original"].drop("clinicalStatus", "stopReason", "PhaseT",'maxDoEArrayN').columns[25:]

['diagonalYes',
 'oneCellYes',
 'hasGeneticEvidence',
 'L2GAndColoc',
 'NoneCellYes',
 'NdiagonalYes',
 'max_L2GScore>=0_1',
 'max_L2GScore>=0_15',
 'max_L2GScore>=0_2',
 'max_L2GScore>=0_25',
 'max_L2GScore>=0_3',
 'max_L2GScore>=0_35',
 'max_L2GScore>=0_4',
 'max_L2GScore>=0_45',
 'max_L2GScore>=0_5',
 'max_L2GScore>=0_55',
 'max_L2GScore>=0_6',
 'max_L2GScore>=0_65',
 'max_L2GScore>=0_7',
 'max_L2GScore>=0_72',
 'max_L2GScore>=0_74',
 'max_L2GScore>=0_75',
 'max_L2GScore>=0_76',
 'max_L2GScore>=0_78',
 'max_L2GScore>=0_8',
 'max_L2GScore>=0_82',
 'max_L2GScore>=0_84',
 'max_L2GScore>=0_85',
 'max_L2GScore>=0_86',
 'max_L2GScore>=0_88',
 'max_L2GScore>=0_9',
 'max_L2GScore>=0_92',
 'max_L2GScore>=0_94',
 'max_L2GScore>=0_95',
 'max_L2GScore>=0_96',
 'max_L2GScore>=0_98',
 'max_L2GScore>=0_1&hasGeneticEvidence_combined',
 'max_L2GScore>=0_15&hasGeneticEvidence_combined',
 'max_L2GScore>=0_2&hasGeneticEvidence_combined',
 'max_L2GScore>=0_25&hasGeneticEvidence_combined',
 'max_L2GScore

In [2]:
aggSetups_original

[Row(comparison='diagonalYes', comparisonType='byDatatype', _1='Phase4', _2='clinical'),
 Row(comparison='diagonalYes', comparisonType='byDatatype', _1='Phase>=3', _2='clinical'),
 Row(comparison='diagonalYes', comparisonType='byDatatype', _1='Phase>=2', _2='clinical'),
 Row(comparison='diagonalYes', comparisonType='byDatatype', _1='Phase>=1', _2='clinical'),
 Row(comparison='diagonalYes', comparisonType='byDatatype', _1='PhaseT', _2='clinical'),
 Row(comparison='oneCellYes', comparisonType='byDatatype', _1='Phase4', _2='clinical'),
 Row(comparison='oneCellYes', comparisonType='byDatatype', _1='Phase>=3', _2='clinical'),
 Row(comparison='oneCellYes', comparisonType='byDatatype', _1='Phase>=2', _2='clinical'),
 Row(comparison='oneCellYes', comparisonType='byDatatype', _1='Phase>=1', _2='clinical'),
 Row(comparison='oneCellYes', comparisonType='byDatatype', _1='PhaseT', _2='clinical'),
 Row(comparison='hasGeneticEvidence', comparisonType='byDatatype', _1='Phase4', _2='clinical'),
 Row(co

In [ ]:

for key, df_analysis in datasetDict.items():
    df_analysis.persist()
    print("original", key)
    for row in aggSetups_original:
        aggregations_original(df_analysis, key, listado, *row, today_date)
    df_analysis.unpersist()
    print("unpersist original", key)

for key, df_analysis in datasetDict_propag.items():
    df_analysis.persist()
    print("original", key)
    for row in aggSetups_original:
        aggregations_original(df_analysis, key, listado, *row, today_date)
    df_analysis.unpersist()
    print("unpersist original", key)

In [ ]:

for key, df_analysis in datasetDict_propag.items():
    df_analysis.persist()
    print("original", key)
    for row in aggSetups_original:
        aggregations_original(df_analysis, key, listado, *row, today_date)
    df_analysis.unpersist()
    print("unpersist original", key)

original max_L2GScore_original


2025-05-07_analysis_L2G/max_L2GScore_original/diagonalYes_Phase4.parquet
2025-05-07 21:46:17.685906


2025-05-07_analysis_L2G/max_L2GScore_original/diagonalYes_Phase>=3.parquet
2025-05-07 21:46:25.581110


2025-05-07_analysis_L2G/max_L2GScore_original/diagonalYes_Phase>=2.parquet
2025-05-07 21:46:33.013601


2025-05-07_analysis_L2G/max_L2GScore_original/diagonalYes_Phase>=1.parquet
2025-05-07 21:46:40.248431


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/miniconda3/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [6]:
df_results

NameError: name 'df_results' is not defined

In [ ]:

for row in aggSetups_original:
    for key, df_analysis in datasetDict.items():
        print("original", key, row)
        aggregations_original(df_analysis, key, listado, *row, today_date)
        df_analysis.persist()
        aggSetups_original = comparisons_df(df_analysis)
        df_analysis.unpersist()
    print("unpersist original", key)


for row in aggSetups_original:
    for key, df_analysis in datasetDict_propag.items():
        print("original", key, row)
        aggregations_original(df_analysis, key, listado, *row, today_date)
        df_analysis.persist()
        aggSetups_original = comparisons_df(df_analysis)
    df_analysis.unpersist()
    print("unpersist original", key)


print("finished analysis")

print("creating pandas dataframe with resulting rows")
df = pd.DataFrame(
    results,
    columns=[
        "type",
        "criteria",
        "phase",
        "OR",
        "pValue",
        "LowCI",
        "HighCI",
        "total",
        "array",
        "rs",
        "lowRs",
        "HighRs",
        "path",
    ],
)

schema = StructType(
    [
        StructField("type", StringType(), True),
        StructField("criteria", StringType(), True),
        StructField("phase", StringType(), True),
        StructField("oddsRatio", DoubleType(), True),
        StructField("pValue", DoubleType(), True),
        StructField("lowerInterval", DoubleType(), True),
        StructField("upperInterval", DoubleType(), True),
        StructField("total", StringType(), True),
        StructField("values", ArrayType(ArrayType(IntegerType())), True),
        StructField("relSuccess", DoubleType(), True),
        StructField("rsLower", DoubleType(), True),
        StructField("rsUpper", DoubleType(), True),
        StructField("path", StringType(), True),
    ]
)

# Convert list of lists to DataFrame
print("created pandas dataframe")
print("converting to spark dataframe")
pattern = r"(\b[0-9]+)_(\d+\b)"  # Matches patterns like 0_1, 0_95, etc.
df = (
    spreadSheetFormatter(spark.createDataFrame(results, schema=schema))
    .withColumn("range", F.regexp_extract(F.col("criteria"), pattern, 0))
    .withColumn(
        "range", F.regexp_replace(F.col("range"), "_", ".")  ## substitute "_" by "."
    )
    .withColumn(
        "comparison",
        F.regexp_extract(F.col("criteria"), r"&(.+)", 1),  # take string after "&"
    )
    .withColumn(
        "comparison",
        F.regexp_replace(F.col("comparison"), "_combined$", ""),  ### trims "combined"
    )
    .withColumn("dimension", F.regexp_extract(F.col("type"), r"_(original|propag)", 1))
)


print("writting dataframe")
df.toPandas().to_csv(
    f"gs://ot-team/jroldan/analysis/{today_date}_newColoc_L2Gbenchmark_4geneticDataAllPhases_newAssoc.csv"
)

print("dataframe written \n Analysis finished")


In [ ]:


for row in aggSetups_original[2:3]:
    for key, df_analysis in datasetDict.items():
        print("original", key, row)
        aggregations_original(df_analysis, key, listado, *row, today_date)
        df_analysis.persist()
        aggSetups_original = comparisons_df(df_analysis)
    df_analysis.unpersist()
    print("unpersist original", key)


for row in aggSetups_original[2:3]:
    for key, df_analysis in datasetDict_propag.items():
        print("original", key, row)
        aggregations_original(df_analysis, key, listado, *row, today_date)
        df_analysis.persist()
        aggSetups_original = comparisons_df(df_analysis)
    df_analysis.unpersist()
    print("unpersist original", key)


print("finished analysis")

print("creating pandas dataframe with resulting rows")
df = pd.DataFrame(
    results,
    columns=[
        "type",
        "criteria",
        "phase",
        "OR",
        "pValue",
        "LowCI",
        "HighCI",
        "total",
        "array",
        "rs",
        "lowRs",
        "HighRs",
        "path",
    ],
)

schema = StructType(
    [
        StructField("type", StringType(), True),
        StructField("criteria", StringType(), True),
        StructField("phase", StringType(), True),
        StructField("oddsRatio", DoubleType(), True),
        StructField("pValue", DoubleType(), True),
        StructField("lowerInterval", DoubleType(), True),
        StructField("upperInterval", DoubleType(), True),
        StructField("total", StringType(), True),
        StructField("values", ArrayType(ArrayType(IntegerType())), True),
        StructField("relSuccess", DoubleType(), True),
        StructField("rsLower", DoubleType(), True),
        StructField("rsUpper", DoubleType(), True),
        StructField("path", StringType(), True),
    ]
)

# Convert list of lists to DataFrame
print("created pandas dataframe")
print("converting to spark dataframe")
pattern = r"(\b[0-9]+)_(\d+\b)"  # Matches patterns like 0_1, 0_95, etc.
df = (
    spreadSheetFormatter(spark.createDataFrame(results, schema=schema))
    .withColumn("range", F.regexp_extract(F.col("criteria"), pattern, 0))
    .withColumn(
        "range", F.regexp_replace(F.col("range"), "_", ".")  ## substitute "_" by "."
    )
    .withColumn(
        "comparison",
        F.regexp_extract(F.col("criteria"), r"&(.+)", 1),  # take string after "&"
    )
    .withColumn(
        "comparison",
        F.regexp_replace(F.col("comparison"), "_combined$", ""),  ### trims "combined"
    )
    .withColumn("dimension", F.regexp_extract(F.col("type"), r"_(original|propag)", 1))
)


print("writting dataframe")
df.toPandas().to_csv(
    f"gs://ot-team/jroldan/analysis/{today_date}_newColoc_L2Gbenchmark_4geneticDataAllPhases_newAssoc.csv"
)

print("dataframe written \n Analysis finished")

In [23]:
dataset=datasetDict["max_L2GScore_original"]
dataset.drop("clinicalStatus", "stopReason", "PhaseT",'maxDoEArrayN').columns[25:31]


['diagonalYes',
 'oneCellYes',
 'hasGeneticEvidence',
 'L2GAndColoc',
 'NoneCellYes',
 'NdiagonalYes']

In [3]:
for row in aggSetups_original[2:3]:
    for key, df_analysis in datasetDict.items():
        print("original", key, row)
        aggregations_original(df_analysis, key, listado, *row, today_date)
        df_analysis.persist()
        aggSetups_original = comparisons_df(df_analysis)
    df_analysis.unpersist()
    print("unpersist original", key)


original max_L2GScore_original Row(comparison='diagonalYes', comparisonType='byDatatype', _1='Phase>=2', _2='clinical')


2025-05-07_analysis/max_L2GScore_original/diagonalYes_Phase>=2.parquet
2025-05-07 20:54:10.620215
original minDistFootPrintMean_original Row(comparison='diagonalYes', comparisonType='byDatatype', _1='Phase>=2', _2='clinical')


2025-05-07_analysis/minDistFootPrintMean_original/diagonalYes_Phase>=2.parquet
2025-05-07 20:54:41.951777
original minTssDistance_original Row(comparison='diagonalYes', comparisonType='byDatatype', _1='Phase>=2', _2='clinical')


2025-05-07_analysis/minTssDistance_original/diagonalYes_Phase>=2.parquet
2025-05-07 20:55:14.146331
unpersist original minTssDistance_original


In [4]:
results

[['max_L2GScore_original',
  'diagonalYes',
  'Phase>=2',
  2.63,
  4.61753289124352e-05,
  1.57,
  4.72,
  '74197',
  [[151, 16], [57907, 16123]],
  1.16,
  1.1,
  1.21,
  'gs://ot-team/jroldan/2025-05-07_analysis/max_L2GScore_original/diagonalYes_Phase>=2.parquet'],
 ['minDistFootPrintMean_original',
  'diagonalYes',
  'Phase>=2',
  2.47,
  0.00021152707847680273,
  1.47,
  4.44,
  '74197',
  [[142, 16], [57916, 16123]],
  1.15,
  1.09,
  1.21,
  'gs://ot-team/jroldan/2025-05-07_analysis/minDistFootPrintMean_original/diagonalYes_Phase>=2.parquet'],
 ['minTssDistance_original',
  'diagonalYes',
  'Phase>=2',
  2.49,
  0.000154067159074883,
  1.48,
  4.47,
  '74197',
  [[143, 16], [57915, 16123]],
  1.15,
  1.09,
  1.21,
  'gs://ot-team/jroldan/2025-05-07_analysis/minTssDistance_original/diagonalYes_Phase>=2.parquet']]

finished analysis
creating pandas dataframe with resulting rows
created pandas dataframe
converting to spark dataframe
importing functions
imported functions
writting dataframe
dataframe written 
 Analysis finished


In [ ]:
datasetDict[f"max_L2GScore_original"].groupBy(
    "max_L2GScore>=0_94&NdiagonalYes_combined",
    "max_L2GScore>=0_94&diagonalYes_combined",
).pivot("phase4").count().show()

+----------------------------------------+---------------------------------------+-----+----+
|max_L2GScore>=0_94&NdiagonalYes_combined|max_L2GScore>=0_94&diagonalYes_combined|   no| yes|
+----------------------------------------+---------------------------------------+-----+----+
|                                     yes|                                     no|   11|   2|
|                                      no|                                     no|67917|6232|
|                                     yes|                                    yes|   15|  20|
+----------------------------------------+---------------------------------------+-----+----+



In [11]:
datasetDict[f"max_L2GScore_original"].columns

['targetId',
 'diseaseId',
 'max_L2GScore',
 'GoF_protect',
 'GoF_risk',
 'LoF_protect',
 'LoF_risk',
 'noEvaluable',
 'coherencyDiagonal',
 'coherencyOneCell',
 'maxClinPhase',
 'coherencyDiagonal_ch',
 'coherencyOneCell_ch',
 'LoF_protect_ch',
 'GoF_protect_ch',
 'diagonalAgreeWithDrugs',
 'oneCellAgreeWithDrugs',
 'arrayN',
 'maxDoE',
 'maxDoE_names',
 'Phase4',
 'Phase>=3',
 'Phase>=2',
 'Phase>=1',
 'Phase0',
 'diagonalYes',
 'oneCellYes',
 'hasGeneticEvidence',
 'L2GAndColoc',
 'maxDoEArrayN',
 'NoneCellYes',
 'NdiagonalYes',
 'stopReason',
 'PhaseT',
 'max_L2GScore>=0_1',
 'max_L2GScore>=0_15',
 'max_L2GScore>=0_2',
 'max_L2GScore>=0_25',
 'max_L2GScore>=0_3',
 'max_L2GScore>=0_35',
 'max_L2GScore>=0_4',
 'max_L2GScore>=0_45',
 'max_L2GScore>=0_5',
 'max_L2GScore>=0_55',
 'max_L2GScore>=0_6',
 'max_L2GScore>=0_65',
 'max_L2GScore>=0_7',
 'max_L2GScore>=0_72',
 'max_L2GScore>=0_74',
 'max_L2GScore>=0_75',
 'max_L2GScore>=0_76',
 'max_L2GScore>=0_78',
 'max_L2GScore>=0_8',
 'max_L

In [1]:
#### BUILDING THE NEW GWAS GENETIC EVIDENCE FROM COLOC
from functions import (
    discrepancifier,
    build_gwasResolvedColoc,
    temporary_directionOfEffect,
    spreadSheetFormatter,
)
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F
import matplotlib.pyplot as plt
from datetime import date, datetime
from pyspark.sql.types import (
    StructType,
    StructField,
    ArrayType,
    DoubleType,
    DecimalType,
    StringType,
    FloatType,
    IntegerType,
)
import pandas as pd

spark = SparkSession.builder.getOrCreate()

# path = "gs://open-targets-pre-data-releases/24.12-uo_test-3/output/etl/parquet/"

path_n='gs://open-targets-data-releases/25.03/output/'


evidences = (
    spark.read.parquet(f"{path_n}evidence").filter(
        F.col("datasourceId").isin(
            [
                "gwas_credible_sets",
                "chembl",
            ]
        )
    )
    # .persist()
)

negativeTD = (
    evidences.filter(F.col("datasourceId") == "chembl")
    .select("targetId", "diseaseId", "studyStopReason", "studyStopReasonCategories")
    .filter(F.array_contains(F.col("studyStopReasonCategories"), "Negative"))
    .groupBy("targetId", "diseaseId")
    .count()
    .withColumn("stopReason", F.lit("Negative"))
    .drop("count")
)

#### include here right way of generating gwasResolvedColoc
#### resolved coloc: 
diseases = spark.read.parquet(f"{path_n}disease/")

credible = spark.read.parquet(f"{path_n}credible_set")

new = spark.read.parquet(f"{path_n}colocalisation_coloc") 

index=spark.read.parquet(f"{path_n}study/")

variantIndex = spark.read.parquet(f"{path_n}variant")

biosample = spark.read.parquet(f"{path_n}biosample")

print("loaded files")

newColoc = (
    new.join(
        credible.selectExpr(  #### studyLocusId from credible set to uncover the codified variants on left side
            "studyLocusId as leftStudyLocusId",
            "StudyId as leftStudyId",
            "variantId as leftVariantId",
            "studyType as credibleLeftStudyType",
        ),
        on="leftStudyLocusId",
        how="left",
    )
    .join(
        credible.selectExpr(  #### studyLocusId from credible set to uncover the codified variants on right side
            "studyLocusId as rightStudyLocusId",
            "studyId as rightStudyId",
            "variantId as rightVariantId",
            "studyType as credibleRightStudyType",
            'isTransQtl'
        ),
        on="rightStudyLocusId",
        how="left",
    )
    .join(
        index.selectExpr(  ### bring modulated target on right side (QTL study)
            "studyId as rightStudyId",
            "geneId",
            "projectId",
            "studyType as indexStudyType",
            "condition",
            "biosampleId",
        ),
        on="rightStudyId",
        how="left",
)
    # .persist()
)

print("loaded newColoc")

# remove columns without content (only null values on them)
df = evidences.filter((F.col("datasourceId") == "gwas_credible_sets"))

# Use an aggregation to determine non-null columns
non_null_counts = df.select(
    *[F.sum(F.col(col).isNotNull().cast("int")).alias(col) for col in df.columns]
)

# Collect the counts for each column
non_null_columns = [
    row[0] for row in non_null_counts.collect()[0].asDict().items() if row[1] > 0
]

# Select only the non-null columns
filtered_df = df.select(*non_null_columns)  # .persist()

## bring studyId, variantId, beta from Gwas and pValue
gwasComplete = filtered_df.join(
    credible.selectExpr(
        "studyLocusId", "studyId", "variantId", "beta as betaGwas", "pValueExponent"
    ),
    on="studyLocusId",
    how="left",
)  # .persist()

print("loaded gwasComplete")

resolvedColoc = (
    (
        newColoc.withColumnRenamed("geneId", "targetId")
        .join(
            gwasComplete.withColumnRenamed("studyLocusId", "leftStudyLocusId"),
            on=["leftStudyLocusId", "targetId"],
            how="inner",
        )
        .join(  ### propagated using parent terms
            diseases.selectExpr(
                "id as diseaseId", "name", "parents", "therapeuticAreas"
            ),
            on="diseaseId",
            how="left",
        )
        .withColumn(
            "diseaseId",
            F.explode_outer(F.concat(F.array(F.col("diseaseId")), F.col("parents"))),
        )
        .drop("parents", "oldDiseaseId")
    ).withColumn(
        "colocDoE",
        F.when(
            F.col("rightStudyType").isin(
                ["eqtl", "pqtl", "tuqtl", "sceqtl", "sctuqtl"]
            ),
            F.when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("GoF_risk"),
            )
            .when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("LoF_risk"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("LoF_protect"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("GoF_protect"),
            ),
        ).when(
            F.col("rightStudyType").isin(
                ["sqtl", "scsqtl"]
            ),  ### opposite directionality than sqtl
            F.when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("LoF_risk"),
            )
            .when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("GoF_risk"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("GoF_protect"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("LoF_protect"),
            ),
        ),
    )
    # .persist()
)
print("loaded resolvedColloc")

##### 


### alternative for gwas credible assoc 
window_spec = Window.partitionBy("targetId", "diseaseId",'leftStudyId').orderBy( ### include gwas study
    F.col("pValueExponent").asc()
)
gwasCredibleAssoc = (
    resolvedColoc.withColumn(
        "homogenized", F.first("colocDoE", ignorenulls=True).over(window_spec)
    )  ## added 30.01.2025
    .select(
        "targetId",
        "diseaseId",
        "homogenized",
        "leftStudyLocusId",
        "h4",
        "datasourceId",
        "resourceScore",
        "leftVariantId",
        "credibleLeftStudyType",
    )
    .withColumn(
        "homogenized",
        F.when(F.col("homogenized").isNull(), F.lit("noEvaluable")).otherwise(
            F.col("homogenized")
        ),
    )
)


#### LOAD STUDYLOCUSID AND VARIANT DISTANCES

l2gPred=spark.read.parquet(f"{path_n}l2g_prediction/")

l2gTable = (
    l2gPred.select("studyLocusId", "geneId", F.explode_outer("features")).select('studyLocusId','geneId','col.name','col.value').filter(
        F.col("col.name").isin(["distanceFootprintMean", "distanceTssMean"])
    )
    .groupBy("studyLocusId", "geneId")
    .pivot("name")
    .agg(F.first("value"))
)

print("creating gwasCredibleAssocDistances")
gwasCredibleAssocDistances = gwasCredibleAssoc.join(
    l2gTable.withColumnRenamed("studyLocusId", "leftStudyLocusId").withColumnRenamed(
        "geneId", "targetId"
    ),
    on=["leftStudyLocusId", "targetId"],
    how="left",
)

print("creating analysis_chembl")

datasource_filter = ["chembl"]
preanalysis_chembl, evidences, actionType, oncolabel = temporary_directionOfEffect(
    path_n, datasource_filter
)
analysis_chembl = (
    discrepancifier(
        preanalysis_chembl.withColumn(
            "maxClinPhase",
            F.max(F.col("clinicalPhase")).over(
                Window.partitionBy("targetId", "diseaseId")
            ),
        )
        .groupBy("targetId", "diseaseId", "maxClinPhase")
        .pivot("homogenized")
        .agg(F.count("targetId"))
        .persist()
    )  #### avoid the
    #    .filter(  ### ensure drug has annotated MoA and is coherent per Target-Disease
    #        ((F.col("GoF_protect").isNotNull()) | F.col("LoF_protect").isNotNull())
    #        & (F.col("coherencyDiagonal") == "coherent")
).selectExpr(
    "targetId",
    "diseaseId",
    "maxClinPhase",
    "coherencyDiagonal as coherencyDiagonal_ch",
    "coherencyOneCell as coherencyOneCell_ch",
    "LoF_protect as LoF_protect_ch",
    "GoF_protect as GoF_protect_ch",
)

### NEWWWWW ### 
doe_columns=["LoF_protect", "GoF_risk", "LoF_risk", "GoF_protect"]
diagonal_lof=['LoF_protect','GoF_risk']
diagonal_gof=['LoF_risk','GoF_protect']

diseases2 = diseases.select("id", "parents").withColumn(
    "diseaseIdPropagated",
    F.explode_outer(F.concat(F.array(F.col("id")), F.col("parents"))),
)
### pivot colocdoE grouping by T-D-studyLocusId-distances

value = "max_L2GScore"

list_l2g = [
    0.10,
    0.15,
    0.20,
    0.25,
    0.30,
    0.35,
    0.40,
    0.45,
    0.50,
    0.55,
    0.60,
    0.65,
    0.70,
    0.72,
    0.74,
    0.75,
    0.76,
    0.78,
    0.80,
    0.82,
    0.84,
    0.85,
    0.86,
    0.88,
    0.90,
    0.92,
    0.94,
    0.95,
    0.96,
    0.98,
]
print("creating benchmarkOT function")

dict_comb = {}


def benchmarkOT(
    dict_comb,
    value,
    gwasCredibleAssocDistances,
    analysis_chembl,
    negativeTD,
    list_l2g,
):
    dict_comb = {
        "hasGeneticEvidence": f"{value}",
        "diagonalYes": f"{value}",
        "oneCellYes": f"{value}",
        "L2GAndColoc": f"{value}",
        'NoneCellYes': f"{value}",
        'NdiagonalYes': f"{value}",
    }
    min_value = gwasCredibleAssocDistances.agg(
        F.min("distanceFootprintMean")
    ).collect()[0][
        0
    ]  ## take min value for scalating
    conditions = [
        F.when(F.col(c) == F.col("maxDoE"), F.lit(c)).otherwise(F.lit(None)) for c in doe_columns
        ]

    dataset_original = discrepancifier(
        gwasCredibleAssocDistances
        # .filter(F.col("h4").isNotNull()) #### not filter by this because we want to include the L2G AND Coloc question
        .withColumn(  ### take maximum L2G score per T-D
            "max_L2GScore",
            F.max("resourceScore").over(Window.partitionBy("targetId", "diseaseId")),
        )
        .withColumn("min_distanceFootprintMean", F.lit(min_value))
        .withColumn(
            "scaled_distanceFootprintMean",
            (F.col("distanceFootprintMean") - F.col("min_distanceFootprintMean"))
            / (1 - F.col("min_distanceFootprintMean")),
        )
        .withColumn(
            "minDistFootPrintMean",
            F.min("scaled_distanceFootprintMean").over(
                Window.partitionBy("targetId", "diseaseId")
            ),
        )
        .withColumn(
            "minTssDistance",
            F.min("distanceTssMean").over(Window.partitionBy("targetId", "diseaseId")),
        )
        .groupBy(
            "targetId",
            "diseaseId",
            f"{value}",
            # "leftStudyLocusId",
        )
        .pivot("homogenized")
        .count()
    )
    dataset_propag = discrepancifier(
        gwasCredibleAssocDistances
        # .filter(F.col("h4").isNotNull()) #### not filter by this because we want to include the L2G AND Coloc question
        .withColumn(  ### take maximum L2G score per T-D
            "max_L2GScore",
            F.max("resourceScore").over(Window.partitionBy("targetId", "diseaseId")),
        )
        .withColumn("min_distanceFootprintMean", F.lit(min_value))
        .withColumn(
            "scaled_distanceFootprintMean",
            (F.col("distanceFootprintMean") - F.col("min_distanceFootprintMean"))
            / (1 - F.col("min_distanceFootprintMean")),
        )
        .withColumn(
            "minDistFootPrintMean",
            F.min("scaled_distanceFootprintMean").over(
                Window.partitionBy("targetId", "diseaseId")
            ),
        )
        .withColumn(
            "minTssDistance",
            F.min("distanceTssMean").over(Window.partitionBy("targetId", "diseaseId")),
        )
        .join(
            diseases2.selectExpr("id as diseaseId", "diseaseIdPropagated"),
            on="diseaseId",
            how="left",
        )
        .withColumnRenamed("diseaseId", "oldDiseaseId")
        .withColumnRenamed("diseaseIdPropagated", "diseaseId")
        .groupBy(
            "targetId",
            "diseaseId",
            f"{value}",
            # "leftStudyLocusId",
        )
        .pivot("homogenized")
        .count()
    )

    return (
        dataset_original.join(
            analysis_chembl, on=["targetId", "diseaseId"], how="right"
        )
        .withColumn(
            "diagonalAgreeWithDrugs",
            F.when(
                (
                    (F.col("coherencyDiagonal_ch") == "coherent")
                    & (F.col("coherencyDiagonal") == "coherent")
                )
                # & (F.col("coherencyDiagonal") == "coherent")
                ,
                F.when(
                    (F.col("LoF_protect_ch").isNotNull())
                    & (
                        F.col("GoF_risk").isNotNull() | F.col("LoF_protect").isNotNull()
                    ),
                    F.lit("coherent"),
                )
                .when(
                    F.col("GoF_protect_ch").isNotNull()
                    & (
                        F.col("LoF_risk").isNotNull() | F.col("GoF_protect").isNotNull()
                    ),
                    F.lit("coherent"),
                )
                .otherwise(F.lit("dispar")),
            ),
        )
        .withColumn(
            "oneCellAgreeWithDrugs",
            F.when(
                (
                    (F.col("coherencyOneCell_ch") == "coherent")
                    & (F.col("coherencyDiagonal") == "coherent")
                )
                # & (F.col("coherencyOneCell") == "coherent")
                ,
                F.when(
                    (F.col("LoF_protect_ch").isNotNull())
                    & (
                        (F.col("LoF_protect").isNotNull())
                        & (F.col("LoF_risk").isNull())
                        & (F.col("GoF_protect").isNull())
                        & (F.col("GoF_risk").isNull())
                    ),
                    F.lit("coherent"),
                )
                .when(
                    (F.col("GoF_protect_ch").isNotNull())
                    & (
                        (F.col("GoF_protect").isNotNull())
                        & (F.col("LoF_risk").isNull())
                        & (F.col("LoF_protect").isNull())
                        & (F.col("GoF_risk").isNull())
                    ),
                    F.lit("coherent"),
                )
                .otherwise(F.lit("dispar")),
            ),
            # ).filter(
            #    F.col("diagonalAgreeWithDrugs").isNotNull()
        ).withColumn(
            "arrayN", F.array(*[F.col(c) for c in doe_columns])
        ).withColumn(
            "maxDoE", F.array_max(F.col("arrayN"))
        ).withColumn("maxDoE_names", F.array(*conditions)
        ).withColumn("maxDoE_names", F.expr("filter(maxDoE_names, x -> x is not null)")
        )
        .withColumn(
            "Phase4",
            F.when(F.col("maxClinPhase") == 4, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "Phase>=3",
            F.when(F.col("maxClinPhase") >= 3, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "Phase>=2",
            F.when(F.col("maxClinPhase") >= 2, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "Phase>=1",
            F.when(F.col("maxClinPhase") >= 1, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "Phase0",
            F.when(F.col("maxClinPhase") == 0, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "diagonalYes",
            F.when(
                F.col("diagonalAgreeWithDrugs") == "coherent", F.lit("yes")
            ).otherwise(F.lit("no")),
        )
        .withColumn(
            "oneCellYes",
            F.when(
                F.col("oneCellAgreeWithDrugs") == "coherent", F.lit("yes")
            ).otherwise(F.lit("no")),
        )
        .withColumn(
            "hasGeneticEvidence",
            F.when(F.col(f"{value}").isNotNull(), F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "L2GAndColoc",
            F.when(
                (F.col(f"{value}").isNotNull())
                & (F.col("coherencyDiagonal").isin(["coherent", "dispar"])),
                F.lit("yes"),
            ).otherwise(F.lit("no")),
        )
        .withColumn(
            "maxDoEArrayN",
            F.expr("aggregate(arrayN, 0, (acc, x) -> acc + IF(x = maxDoE, 1, 0))")
        ).withColumn(
            "NoneCellYes",
            F.when(F.col("LoF_protect_ch").isNotNull() & (F.array_contains(F.col("maxDoE_names"), F.lit("LoF_protect")))==True, F.lit('yes'))
            .when(F.col("GoF_protect_ch").isNotNull() & (F.array_contains(F.col("maxDoE_names"), F.lit("GoF_protect")))==True, F.lit('yes')
                ).otherwise(F.lit('no'))  # If the value is null, return null # Otherwise, check if name is in array
        ).withColumn(
            "NdiagonalYes",
            F.when(F.col("LoF_protect_ch").isNotNull() & 
                (F.size(F.array_intersect(F.col("maxDoE_names"), F.array([F.lit(x) for x in diagonal_lof]))) > 0),
                F.lit("yes")
            ).when(F.col("GoF_protect_ch").isNotNull() & 
                (F.size(F.array_intersect(F.col("maxDoE_names"), F.array([F.lit(x) for x in diagonal_gof]))) > 0),
                F.lit("yes")
            ).otherwise(F.lit('no'))
        )
        .join(negativeTD, on=["targetId", "diseaseId"], how="left")
        .withColumn(
            "PhaseT",
            F.when(F.col("stopReason") == "Negative", F.lit("yes")).otherwise(F.lit("no")),
        )
        .select(
            ["*"]
            + (
                [  ### single columns
                    F.when(F.col(f"{value}") >= n, F.lit("yes"))
                    .otherwise(F.lit("no"))
                    .alias(f"{value}>={str(n).replace('.', '_')}")
                    for n in list_l2g
                ]
            )
            + (
                [  ### column combinations for Yes/No colums Plus has DoE (any agreement)
                    F.when((F.col(a) == "yes") & (F.col(x) >= n), F.lit("yes"))
                    .otherwise(F.lit("no"))
                    .alias(f"{x}>={str(n).replace('.', '_')}&{a}_combined")
                    for a, x in dict_comb.items()
                    for n in list_l2g
                ]
            )
        ),
        dataset_propag.join(
            analysis_chembl, on=["targetId", "diseaseId"], how="right"
        )
        .withColumn(
            "diagonalAgreeWithDrugs",
            F.when(
                (
                    (F.col("coherencyDiagonal_ch") == "coherent")
                    & (F.col("coherencyDiagonal") == "coherent")
                )
                # & (F.col("coherencyDiagonal") == "coherent")
                ,
                F.when(
                    (F.col("LoF_protect_ch").isNotNull())
                    & (
                        F.col("GoF_risk").isNotNull() | F.col("LoF_protect").isNotNull()
                    ),
                    F.lit("coherent"),
                )
                .when(
                    F.col("GoF_protect_ch").isNotNull()
                    & (
                        F.col("LoF_risk").isNotNull() | F.col("GoF_protect").isNotNull()
                    ),
                    F.lit("coherent"),
                )
                .otherwise(F.lit("dispar")),
            ),
        )
        .withColumn(
            "oneCellAgreeWithDrugs",
            F.when(
                (
                    (F.col("coherencyOneCell_ch") == "coherent")
                    & (F.col("coherencyDiagonal") == "coherent")
                )
                # & (F.col("coherencyOneCell") == "coherent")
                ,
                F.when(
                    (F.col("LoF_protect_ch").isNotNull())
                    & (
                        (F.col("LoF_protect").isNotNull())
                        & (F.col("LoF_risk").isNull())
                        & (F.col("GoF_protect").isNull())
                        & (F.col("GoF_risk").isNull())
                    ),
                    F.lit("coherent"),
                )
                .when(
                    (F.col("GoF_protect_ch").isNotNull())
                    & (
                        (F.col("GoF_protect").isNotNull())
                        & (F.col("LoF_risk").isNull())
                        & (F.col("LoF_protect").isNull())
                        & (F.col("GoF_risk").isNull())
                    ),
                    F.lit("coherent"),
                )
                .otherwise(F.lit("dispar")),
            ),
            # ).filter(
            #    F.col("diagonalAgreeWithDrugs").isNotNull()
        ).withColumn(
            "arrayN", F.array(*[F.col(c) for c in doe_columns])
        ).withColumn(
            "maxDoE", F.array_max(F.col("arrayN"))
        ).withColumn("maxDoE_names", F.array(*conditions)
        ).withColumn("maxDoE_names", F.expr("filter(maxDoE_names, x -> x is not null)")
        )
        .withColumn(
            "Phase4",
            F.when(F.col("maxClinPhase") == 4, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "Phase>=3",
            F.when(F.col("maxClinPhase") >= 3, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "Phase>=2",
            F.when(F.col("maxClinPhase") >= 2, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "Phase>=1",
            F.when(F.col("maxClinPhase") >= 1, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "Phase0",
            F.when(F.col("maxClinPhase") == 0, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "diagonalYes",
            F.when(
                F.col("diagonalAgreeWithDrugs") == "coherent", F.lit("yes")
            ).otherwise(F.lit("no")),
        )
        .withColumn(
            "oneCellYes",
            F.when(
                F.col("oneCellAgreeWithDrugs") == "coherent", F.lit("yes")
            ).otherwise(F.lit("no")),
        )
        .withColumn(
            "hasGeneticEvidence",
            F.when(F.col(f"{value}").isNotNull(), F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "L2GAndColoc",
            F.when(
                (F.col(f"{value}").isNotNull())
                & (F.col("coherencyDiagonal").isin(["coherent", "dispar"])),
                F.lit("yes"),
            ).otherwise(F.lit("no")),
        )
        .withColumn(
            "maxDoEArrayN",
            F.expr("aggregate(arrayN, 0, (acc, x) -> acc + IF(x = maxDoE, 1, 0))")
        ).withColumn(
            "NoneCellYes",
            F.when(F.col("LoF_protect_ch").isNotNull() & (F.array_contains(F.col("maxDoE_names"), F.lit("LoF_protect")))==True, F.lit('yes'))
            .when(F.col("GoF_protect_ch").isNotNull() & (F.array_contains(F.col("maxDoE_names"), F.lit("GoF_protect")))==True, F.lit('yes')
                ).otherwise(F.lit('no'))  # If the value is null, return null # Otherwise, check if name is in array
        ).withColumn(
            "NdiagonalYes",
            F.when(F.col("LoF_protect_ch").isNotNull() & 
                (F.size(F.array_intersect(F.col("maxDoE_names"), F.array([F.lit(x) for x in diagonal_lof]))) > 0),
                F.lit("yes")
            ).when(F.col("GoF_protect_ch").isNotNull() & 
                (F.size(F.array_intersect(F.col("maxDoE_names"), F.array([F.lit(x) for x in diagonal_gof]))) > 0),
                F.lit("yes")
            ).otherwise(F.lit('no'))
        )
        .join(negativeTD, on=["targetId", "diseaseId"], how="left")
        .withColumn(
            "PhaseT",
            F.when(F.col("stopReason") == "Negative", F.lit("yes")).otherwise(F.lit("no")),
        )
        .select(
            ["*"]
            + (
                [  ### single columns
                    F.when(F.col(f"{value}") >= n, F.lit("yes"))
                    .otherwise(F.lit("no"))
                    .alias(f"{value}>={str(n).replace('.', '_')}")
                    for n in list_l2g
                ]
            )
            + (
                [  ### column combinations for Yes/No colums Plus has DoE (any agreement)
                    F.when((F.col(a) == "yes") & (F.col(x) >= n), F.lit("yes"))
                    .otherwise(F.lit("no"))
                    .alias(f"{x}>={str(n).replace('.', '_')}&{a}_combined")
                    for a, x in dict_comb.items()
                    for n in list_l2g
                ]
            )
        )
    )

print("creating dataframes in loop")

values = [
    "max_L2GScore",
    "minDistFootPrintMean",
    "minTssDistance",
]

datasetDict = {}
datasetDict_propag = {}

for value in values:
    if value == "max_L2GScore":
        (
            datasetDict[f"max_L2GScore_original"],
            datasetDict_propag[f"max_L2GScore_propag"],
        ) = benchmarkOT(
            dict_comb,
            value,
            gwasCredibleAssocDistances,
            analysis_chembl,
            negativeTD,
            list_l2g,
        )
    else:
        datasetDict[f"{value}_original"], datasetDict_propag[f"{value}_propag"] = (
            benchmarkOT(
                dict_comb,
                value,
                gwasCredibleAssocDistances,
                analysis_chembl,
                negativeTD,
                list_l2g,
            )
        )



def comparisons_df(dataset) -> list:
    """Return list of all comparisons to be used in the analysis"""
    toAnalysis = dataset.drop("clinicalStatus", "stopReason", "PhaseT",'maxDoEArrayN').columns[25:]
    dataType = ["byDatatype"] * len(toAnalysis)
    l_studies = []
    l_studies.extend([list(a) for a in zip(toAnalysis, dataType)])

    schema = StructType(
        [
            StructField("comparison", StringType(), True),
            StructField("comparisonType", StringType(), True),
        ]
    )

    comparisons = spark.createDataFrame(l_studies, schema=schema)
    ### include all the columns as predictor

    predictions = spark.createDataFrame(
        data=[
            ("Phase4", "clinical"),
            ("Phase>=3", "clinical"),
            ("Phase>=2", "clinical"),
            ("Phase>=1", "clinical"),
            ("PhaseT", "clinical"),
        ]
    )
    return comparisons.join(predictions, how="full").collect()


from functions import relative_success, spreadSheetFormatter, convertTuple

full_data = spark.createDataFrame(
    data=[
        ("yes", "yes"),
        ("yes", "no"),
        ("no", "yes"),
        ("no", "no"),
    ],
    schema=StructType(
        [
            StructField("prediction", StringType(), True),
            StructField("comparison", StringType(), True),
        ]
    ),
)


def aggregations_original(
    df,
    data,
    listado,
    comparisonColumn,
    comparisonType,
    predictionColumn,
    predictionType,
    today_date,
):

    wComparison = Window.partitionBy(F.col(comparisonColumn))
    wPrediction = Window.partitionBy(F.col(predictionColumn))
    wPredictionComparison = Window.partitionBy(
        F.col(comparisonColumn), F.col(predictionColumn)
    )

    uniqIds = df.select("targetId", "diseaseId").distinct().count()
    out = (
        df.withColumn("comparisonType", F.lit(comparisonType))
        .withColumn("predictionType", F.lit(predictionType))
        .withColumn("total", F.lit(uniqIds))
        .withColumn("a", F.count("targetId").over(wPredictionComparison))
        .withColumn(
            "predictionTotal",
            F.count("targetId").over(wPrediction),
        )
        .withColumn(
            "comparisonTotal",
            F.count("targetId").over(wComparison),
        )
        .select(
            F.col(predictionColumn).alias("prediction"),
            F.col(comparisonColumn).alias("comparison"),
            "comparisonType",
            "predictionType",
            "a",
            "predictionTotal",
            "comparisonTotal",
            "total",
        )
        .filter(F.col("prediction").isNotNull())
        .filter(F.col("comparison").isNotNull())
        .distinct()
    )
    out.write.mode("overwrite").parquet(
        "gs://ot-team/jroldan/"
        + str(
            today_date
            + "_"
            + "analysis_L2G/"
            + data
            # + "_propagated"
            + "/"
            + comparisonColumn
            + "_"
            + predictionColumn
            + ".parquet"
        )
    )
    filePath = "gs://ot-team/jroldan/" + str(
        today_date
        + "_"
        + "analysis_L2G/"
        + data
        # + "_propagated"
        + "/"
        + comparisonColumn
        + "_"
        + predictionColumn
        + ".parquet"
    )

    listado.append(
        "gs://ot-team/jroldan/"
        + str(
            today_date
            + "_"
            + "analysis_L2G/"
            + data
            # + "_propagated"
            + "/"
            + comparisonColumn
            + "_"
            + predictionColumn
            + ".parquet"
        )
    )
    print(
        today_date
        + "_"
        + "analysis_L2G/"
        + data
        # + "_propagated"
        + "/"
        + comparisonColumn
        + "_"
        + predictionColumn
        + ".parquet"
    )

    c = datetime.now()
    c.strftime("%H:%M:%S")
    print(c)
    array1 = np.delete(
        out.join(full_data, on=["prediction", "comparison"], how="outer")
        .groupBy("comparison")
        .pivot("prediction")
        .agg(F.first("a"))
        .sort(F.col("comparison").desc())
        .select("comparison", "yes", "no")
        .fillna(0)
        .toPandas()
        .to_numpy(),
        [0],
        1,
    )
    total = np.sum(array1)
    res_npPhaseX = np.array(array1, dtype=int)
    resX = convertTuple(fisher_exact(res_npPhaseX, alternative="two-sided"))
    resx_CI = convertTuple(
        odds_ratio(res_npPhaseX).confidence_interval(confidence_level=0.95)
    )

    result_st.append(resX)
    result_ci.append(resx_CI)
    (rs_result, rs_ci) = relative_success(array1)

    results.append(
        [
            data,
            comparisonColumn,
            predictionColumn,
            round(float(resX.split(",")[0]), 2),
            float(resX.split(",")[1]),
            round(float(resx_CI.split(",")[0]), 2),
            round(float(resx_CI.split(",")[1]), 2),
            str(total),
            np.array(res_npPhaseX).tolist(),
            round(float(rs_result), 2),
            round(float(rs_ci[0]), 2),
            round(float(rs_ci[1]), 2),
            filePath,
        ]
    )
    return results


import pandas as pd
import numpy as np
from scipy.stats import fisher_exact
from scipy.stats.contingency import odds_ratio

# Initialize an empty list to store the results
result_st = []
result_ci = []
results = []


def convertTuple(tup):
    st = ",".join(map(str, tup))
    return st


print("launched function to run analysis")


listado = []
today_date = str(date.today())
aggSetups_original = comparisons_df(datasetDict["max_L2GScore_original"])

loaded files
loaded newColoc


25/05/08 04:53:32 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


loaded gwasComplete
loaded resolvedColloc


creating gwasCredibleAssocDistances
creating analysis_chembl


creating benchmarkOT function
creating dataframes in loop


launched function to run analysis


In [2]:
aggSetups_original

[Row(comparison='diagonalYes', comparisonType='byDatatype', _1='Phase4', _2='clinical'),
 Row(comparison='diagonalYes', comparisonType='byDatatype', _1='Phase>=3', _2='clinical'),
 Row(comparison='diagonalYes', comparisonType='byDatatype', _1='Phase>=2', _2='clinical'),
 Row(comparison='diagonalYes', comparisonType='byDatatype', _1='Phase>=1', _2='clinical'),
 Row(comparison='diagonalYes', comparisonType='byDatatype', _1='PhaseT', _2='clinical'),
 Row(comparison='oneCellYes', comparisonType='byDatatype', _1='Phase4', _2='clinical'),
 Row(comparison='oneCellYes', comparisonType='byDatatype', _1='Phase>=3', _2='clinical'),
 Row(comparison='oneCellYes', comparisonType='byDatatype', _1='Phase>=2', _2='clinical'),
 Row(comparison='oneCellYes', comparisonType='byDatatype', _1='Phase>=1', _2='clinical'),
 Row(comparison='oneCellYes', comparisonType='byDatatype', _1='PhaseT', _2='clinical'),
 Row(comparison='hasGeneticEvidence', comparisonType='byDatatype', _1='Phase4', _2='clinical'),
 Row(co

In [ ]:
aggSetups_original

In [5]:
aggSetups_original

[Row(comparison='diagonalYes', comparisonType='byDatatype', _1='Phase4', _2='clinical'),
 Row(comparison='diagonalYes', comparisonType='byDatatype', _1='Phase>=3', _2='clinical'),
 Row(comparison='diagonalYes', comparisonType='byDatatype', _1='Phase>=2', _2='clinical'),
 Row(comparison='diagonalYes', comparisonType='byDatatype', _1='Phase>=1', _2='clinical'),
 Row(comparison='diagonalYes', comparisonType='byDatatype', _1='PhaseT', _2='clinical'),
 Row(comparison='oneCellYes', comparisonType='byDatatype', _1='Phase4', _2='clinical'),
 Row(comparison='oneCellYes', comparisonType='byDatatype', _1='Phase>=3', _2='clinical'),
 Row(comparison='oneCellYes', comparisonType='byDatatype', _1='Phase>=2', _2='clinical'),
 Row(comparison='oneCellYes', comparisonType='byDatatype', _1='Phase>=1', _2='clinical'),
 Row(comparison='oneCellYes', comparisonType='byDatatype', _1='PhaseT', _2='clinical'),
 Row(comparison='hasGeneticEvidence', comparisonType='byDatatype', _1='Phase4', _2='clinical'),
 Row(co

In [4]:
comparisons_df(datasetDict["minTssDistance_original"])

[Row(comparison='diagonalYes', comparisonType='byDatatype', _1='Phase4', _2='clinical'),
 Row(comparison='diagonalYes', comparisonType='byDatatype', _1='Phase>=3', _2='clinical'),
 Row(comparison='diagonalYes', comparisonType='byDatatype', _1='Phase>=2', _2='clinical'),
 Row(comparison='diagonalYes', comparisonType='byDatatype', _1='Phase>=1', _2='clinical'),
 Row(comparison='diagonalYes', comparisonType='byDatatype', _1='PhaseT', _2='clinical'),
 Row(comparison='oneCellYes', comparisonType='byDatatype', _1='Phase4', _2='clinical'),
 Row(comparison='oneCellYes', comparisonType='byDatatype', _1='Phase>=3', _2='clinical'),
 Row(comparison='oneCellYes', comparisonType='byDatatype', _1='Phase>=2', _2='clinical'),
 Row(comparison='oneCellYes', comparisonType='byDatatype', _1='Phase>=1', _2='clinical'),
 Row(comparison='oneCellYes', comparisonType='byDatatype', _1='PhaseT', _2='clinical'),
 Row(comparison='hasGeneticEvidence', comparisonType='byDatatype', _1='Phase4', _2='clinical'),
 Row(co

In [1]:
""" Build datasets with DoE evidence and assoc numbers"""

#from DoEAssessment import directionOfEffect
from functions import temporary_directionOfEffect,build_gwasResolvedColoc_noPropag
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F
from pyspark.ml.feature import QuantileDiscretizer
from datetime import date
from doeFunction import doeFunction

spark = SparkSession.builder.getOrCreate()
today_date = str(date.today())

#platform_v = "25.03"
path = 'gs://open-targets-data-releases/25.03/output/'

replacement_dict = {
    "gene_burden": "GeneBurden",
    "chembl": "ChEMBL",
    "intogen": "Intogen",
    "orphanet": "Orphanet",
    "cancer_gene_census": "CancerGeneCensus",
    "eva": "EvaGermline",
    "gene2phenotype": "Gene2Phenotype",
    "eva_somatic": "EvaSomatic",
    "ot_genetics_portal": "OtGenetics",
    "impc": "IMPC",
    "gwas_credible_set" :"credibleOtGenetics"   
}

doe_sources = [
    #"ot_genetics_portal",
    "gene_burden",
    "eva",
    "eva_somatic",
    "gene2phenotype",
    "orphanet",
    "cancer_gene_census",
    "intogen",
    "impc",
    "chembl",
]

new = spark.read.parquet(f"{path}colocalisation_coloc") 
credible = spark.read.parquet(f"{path}credible_set")
index=spark.read.parquet(f"{path}study/")
evidences = spark.read.parquet(f"{path}evidence")
diseases = spark.read.parquet(f"{path}disease/")

### take only the ones with datasources for DoE
evidences = evidences.filter(F.col("datasourceId").isin(doe_sources))

gwasCredibleAssoc,assessment=doeFunction(new,credible,index,evidences,diseases,path)

gwasCredibleAssoc.withColumn(
        "homogenized",
        F.when(F.col("homogenized").isNull(), F.lit("noEvaluable")).otherwise(
            F.col("homogenized")
        ),
    ).withColumn("variantEffect", 
                F.when(F.col("homogenized").isin(["LoF_protect","LoF_risk"]), F.lit("LoF")
                ).when(F.col("homogenized").isin(["GoF_protect","GoF_risk"]), F.lit("GoF")
                ).otherwise(F.lit("noEvaluable"))
    ).withColumn("directionOnTrait", 
                F.when(F.col("homogenized").isin(["LoF_protect","GoF_protect"]), F.lit("protect")
                ).when(F.col("homogenized").isin(["LoF_risk","GoF_risk"]), F.lit("risk")
                ).otherwise(F.lit("noEvaluable"))
    ).select("targetId", "diseaseId", "homogenized","variantEffect","directionOnTrait")


### replace     
assessment_all = assessment.unionByName(
    gwasCredibleAssoc.withColumn("datasourceId", F.lit("gwas_credible_set")),
    allowMissingColumns=True,
    ).withColumn("niceName", F.col("datasourceId")
    ).replace(replacement_dict, subset=["niceName"]
    ).withColumn("datasourceAll", F.lit("All"))

print("asessment done")

def datasets_numbers_trait(assessment_all, buckets_number):
    """This function creates in a long format (suitable for R) the N of evidences and association per
    DoE section. At the end, it creates a column with the corresponding deciles of the numbers to plot their intesntiy
    The deciles are trained using the assoc to avoid underrating numbers from the assoc respecting the evidences
    """
    """ 
        usage of temporary_directionOfEffect:
        path = "gs://open-targets-pre-data-releases/24.12-uo_test-3/output/etl/parquet/"

        datasource_filter = [
            "ot_genetics_portal",
            "gwas_credible_sets",
            "gene_burden",
            "eva",
            "eva_somatic",
            "gene2phenotype",
            "orphanet",
            "cancer_gene_census",
            "intogen",
            "impc",
            "chembl",
        ]

        assessment, evidences, actionType, oncolabel = temporary_directionOfEffect(
            path, datasource_filter
        )
    """

    ## direction on trait
    unpivot_trait = "stack(2, 'protect', protect, 'risk', risk) as (type, count)"
    trait = (
        assessment_all.filter(F.col("directionOnTrait") != "noEvaluable")
        .groupBy("niceName")
        .pivot("directionOnTrait")
        .count()
        .union(
            assessment_all.filter(F.col("directionOnTrait") != "noEvaluable")
            .groupBy("datasourceAll")
            .pivot("directionOnTrait")
            .count()
            .withColumnRenamed("datasourceAll", "niceName")
        )
        .select("niceName", F.expr(unpivot_trait))
        .fillna(0)
    ).withColumn("facet", F.lit("trait"))

    #### direction on target
    unpivot_function = "stack(2, 'gof', gof, 'lof', lof) as (type, count)"

    function = (
        assessment_all.filter(F.col("variantEffect") != "noEvaluable")
        .groupBy("niceName")
        .pivot("variantEffect")
        .count()
        .union(
            assessment_all.filter(F.col("variantEffect") != "noEvaluable")
            .groupBy("datasourceAll")
            .pivot("variantEffect")
            .count()
            .withColumnRenamed("datasourceAll", "niceName")
        )
        .select("niceName", F.expr(unpivot_function))
        .fillna(0)
    ).withColumn("facet", F.lit("function"))

    #### direction complete
    unpivot_whole = "stack(4, 'LoF_protect', LoF_protect, 'LoF_risk', LoF_risk,'GoF_protect',GoF_protect,'GoF_risk',GoF_risk) as (type, count)"

    whole = (
        assessment_all.filter(F.col("homogenized") != "noEvaluable")
        .groupBy("targetId", "diseaseId", "niceName", "homogenized")
        .count()
        .groupBy("niceName")
        .pivot("homogenized")
        .count()
        .union(
            assessment_all.filter(F.col("homogenized") != "noEvaluable")
            .groupBy("targetId", "diseaseId", "datasourceAll", "homogenized")
            .count()
            .groupBy("datasourceAll")
            .pivot("homogenized")
            .count()
            .withColumnRenamed("datasourceAll", "niceName")
        )
        .select("niceName", F.expr(unpivot_whole))
        .fillna(0)
    ).withColumn("facet", F.lit("whole"))

    qds = QuantileDiscretizer(
        numBuckets=buckets_number,
        inputCol="count",
        outputCol="deciles",
    )

    all = function.union(trait).union(whole)
    result = qds.fit(whole).transform(all)  ### train qds in Whole and transform all
    
    return result

print("read function")

print("executing function")

dataset=datasets_numbers_trait(assessment_all, 11)
print("executed function")
print("printing dataset")
dataset.toPandas().to_csv(f"gs://ot-team/jroldan/analysis/{today_date}_numbersDoeTable.csv")
print("dataset saved succesfully")

loaded newColoc


25/05/10 09:19:31 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


TypeError: '>' not supported between instances of 'NoneType' and 'int'

In [9]:

path='gs://open-targets-data-releases/25.03/output/'

new = spark.read.parquet(f"{path}colocalisation_coloc") 
credible = spark.read.parquet(f"{path}credible_set")
index=spark.read.parquet(f"{path}study/")
evidences = spark.read.parquet(f"{path}evidence")
diseases = spark.read.parquet(f"{path}disease/")

In [10]:
from functions import (
temporary_directionOfEffect,
)
path='gs://open-targets-data-releases/25.03/output/'
import pyspark.sql.functions as F
newColoc = (
    new.join(
        credible.selectExpr(  #### studyLocusId from credible set to uncover the codified variants on left side
            "studyLocusId as leftStudyLocusId",
            "StudyId as leftStudyId",
            "variantId as leftVariantId",
            "studyType as credibleLeftStudyType",
        ),
        on="leftStudyLocusId",
        how="left",
    )
    .join(
        credible.selectExpr(  #### studyLocusId from credible set to uncover the codified variants on right side
            "studyLocusId as rightStudyLocusId",
            "studyId as rightStudyId",
            "variantId as rightVariantId",
            "studyType as credibleRightStudyType",
            'isTransQtl'
        ),
        on="rightStudyLocusId",
        how="left",
    )
    .join(
        index.selectExpr(  ### bring modulated target on right side (QTL study)
            "studyId as rightStudyId",
            "geneId",
            "projectId",
            "studyType as indexStudyType",
            "condition",
            "biosampleId",
        ),
        on="rightStudyId",
        how="left",
)
    # .persist()
)

print("loaded newColoc")

# remove columns without content (only null values on them)
df = evidences.filter((F.col("datasourceId") == "gwas_credible_set"))

# Use an aggregation to determine non-null columns
non_null_counts = df.select(
    *[F.sum(F.col(col).isNotNull().cast("int")).alias(col) for col in df.columns]
)

# Collect the counts for each column
non_null_columns = [
    row[0] for row in non_null_counts.collect()[0].asDict().items() if row[1] > 0
]

# Select only the non-null columns
filtered_df = df.select(*non_null_columns)  # .persist()

## bring studyId, variantId, beta from Gwas and pValue
gwasComplete = filtered_df.join(
    credible.selectExpr(
        "studyLocusId", "studyId", "variantId", "beta as betaGwas", "pValueExponent"
    ),
    on="studyLocusId",
    how="left",
)  # .persist()

print("loaded gwasComplete")

resolvedColoc = (
    (
        newColoc.withColumnRenamed("geneId", "targetId")
        .join(
            gwasComplete.withColumnRenamed("studyLocusId", "leftStudyLocusId"),
            on=["leftStudyLocusId", "targetId"],
            how="inner",
        )
        .join(  ### propagated using parent terms
            diseases.selectExpr(
                "id as diseaseId", "name", "parents", "therapeuticAreas"
            ),
            on="diseaseId",
            how="left",
        )
        .withColumn(
            "diseaseId",
            F.explode_outer(F.concat(F.array(F.col("diseaseId")), F.col("parents"))),
        )
        .drop("parents", "oldDiseaseId")
    ).withColumn(
        "colocDoE",
        F.when(
            F.col("rightStudyType").isin(
                ["eqtl", "pqtl", "tuqtl", "sceqtl", "sctuqtl"]
            ),
            F.when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("GoF_risk"),
            )
            .when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("LoF_risk"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("LoF_protect"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("GoF_protect"),
            ),
        ).when(
            F.col("rightStudyType").isin(
                ["sqtl", "scsqtl"]
            ),  ### opposite directionality than sqtl
            F.when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("LoF_risk"),
            )
            .when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("GoF_risk"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("GoF_protect"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("LoF_protect"),
            ),
        ),
    )
    # .persist()
)
print("loaded resolvedColloc")

gwasCredibleAssoc = (
resolvedColoc.withColumn(
    "homogenized", F.first("colocDoE", ignorenulls=True).over(window_spec)
)  ## added 30.01.2025
.select("targetId", "diseaseId",'leftStudyId', "homogenized")
.withColumn(
    "homogenized",
    F.when(F.col("homogenized").isNull(), F.lit("noEvaluable")).otherwise(
        F.col("homogenized")
    ),
)
)

datasource_filter = [
#"gwas_credible_set", remove so avoid potential duplicates as it will be incorporated later (DoE is done separately)
"gene_burden",
"eva",
"eva_somatic",
"gene2phenotype",
"orphanet",
"cancer_gene_census",
"intogen",
"impc",
"chembl",
]

assessment, evidences, actionType, oncolabel = temporary_directionOfEffect(
path, datasource_filter
)

loaded newColoc


TypeError: '>' not supported between instances of 'NoneType' and 'int'

In [19]:
# remove columns without content (only null values on them)
df = evidences.filter((F.col("datasourceId") == "gwas_credible_sets"))

# Use an aggregation to determine non-null columns
non_null_counts = df.select(
    *[F.sum(F.col(col).isNotNull().cast("int")).alias(col) for col in df.columns]
)


# Collect the counts for each column
non_null_columns = [
    row[0] for row in non_null_counts.collect()[0].asDict().items() if row[1] > 0
]

In [18]:
df.show()

+------------------+---------------+-------------+-------------------+--------+----------+----+---------------------------+---------------------------+---------------------------------+--------------------------------+-----------------+-------------+----------+--------------------+--------+-------------+---------------------+--------------+-----------------+--------+----------------+---------------+----------+--------+-------------------+-------------------+----------------+-----------------+-------------------+-------------------------+-------------------------------------+-------------------------------------+--------------+------+------------+-----------------+----------+----------------------------+-------------------+--------------+---------+--------------------------------+--------------------------------+--------------+--------------+--------+---------+----------+------------+-----------+--------------+-------------------+----+------------------------+-----------------+---------

In [ ]:
df = evidences.filter((F.col("datasourceId") == "gwas_credible_set"))

In [11]:
evidences.filter(F.col('datasourceId')=='gwas_credible_sets').show()

+------------------+---------------+-------------+-------------------+--------+----------+----+---------------------------+---------------------------+---------------------------------+--------------------------------+-----------------+-------------+----------+--------------------+--------+-------------+---------------------+--------------+-----------------+--------+----------------+---------------+----------+--------+-------------------+-------------------+----------------+-----------------+-------------------+-------------------------+-------------------------------------+-------------------------------------+--------------+------+------------+-----------------+----------+----------------------------+-------------------+--------------+---------+--------------------------------+--------------------------------+--------------+--------------+--------+---------+----------+------------+-----------+--------------+-------------------+----+------------------------+-----------------+---------

In [3]:
path='gs://open-targets-data-releases/25.03/output/'

new = spark.read.parquet(f"{path}colocalisation_coloc") 
credible = spark.read.parquet(f"{path}credible_set")
index=spark.read.parquet(f"{path}study/")
evidences = spark.read.parquet(f"{path}evidence")
diseases = spark.read.parquet(f"{path}disease/")

from functions import (
temporary_directionOfEffect,
)
import pyspark.sql.functions as F
newColoc = (
    new.join(
        credible.selectExpr(  #### studyLocusId from credible set to uncover the codified variants on left side
            "studyLocusId as leftStudyLocusId",
            "StudyId as leftStudyId",
            "variantId as leftVariantId",
            "studyType as credibleLeftStudyType",
        ),
        on="leftStudyLocusId",
        how="left",
    )
    .join(
        credible.selectExpr(  #### studyLocusId from credible set to uncover the codified variants on right side
            "studyLocusId as rightStudyLocusId",
            "studyId as rightStudyId",
            "variantId as rightVariantId",
            "studyType as credibleRightStudyType",
            'isTransQtl'
        ),
        on="rightStudyLocusId",
        how="left",
    )
    .join(
        index.selectExpr(  ### bring modulated target on right side (QTL study)
            "studyId as rightStudyId",
            "geneId",
            "projectId",
            "studyType as indexStudyType",
            "condition",
            "biosampleId",
        ),
        on="rightStudyId",
        how="left",
)
    # .persist()
)

print("loaded newColoc")

# remove columns without content (only null values on them)
df = evidences.filter((F.col("datasourceId") == "gwas_credible_sets"))

# Use an aggregation to determine non-null columns
non_null_counts = df.select(
    *[F.sum(F.col(col).isNotNull().cast("int")).alias(col) for col in df.columns]
)

# Collect the counts for each column
non_null_columns = [
    row[0] for row in non_null_counts.collect()[0].asDict().items() if row[1] > 0
]

# Select only the non-null columns
filtered_df = df.select(*non_null_columns)  # .persist()

## bring studyId, variantId, beta from Gwas and pValue
gwasComplete = filtered_df.join(
    credible.selectExpr(
        "studyLocusId", "studyId", "variantId", "beta as betaGwas", "pValueExponent"
    ),
    on="studyLocusId",
    how="left",
)  # .persist()

print("loaded gwasComplete")

resolvedColoc = (
    (
        newColoc.withColumnRenamed("geneId", "targetId")
        .join(
            gwasComplete.withColumnRenamed("studyLocusId", "leftStudyLocusId"),
            on=["leftStudyLocusId", "targetId"],
            how="inner",
        )
        .join(  ### propagated using parent terms
            diseases.selectExpr(
                "id as diseaseId", "name", "parents", "therapeuticAreas"
            ),
            on="diseaseId",
            how="left",
        )
        .withColumn(
            "diseaseId",
            F.explode_outer(F.concat(F.array(F.col("diseaseId")), F.col("parents"))),
        )
        .drop("parents", "oldDiseaseId")
    ).withColumn(
        "colocDoE",
        F.when(
            F.col("rightStudyType").isin(
                ["eqtl", "pqtl", "tuqtl", "sceqtl", "sctuqtl"]
            ),
            F.when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("GoF_risk"),
            )
            .when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("LoF_risk"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("LoF_protect"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("GoF_protect"),
            ),
        ).when(
            F.col("rightStudyType").isin(
                ["sqtl", "scsqtl"]
            ),  ### opposite directionality than sqtl
            F.when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("LoF_risk"),
            )
            .when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("GoF_risk"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("GoF_protect"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("LoF_protect"),
            ),
        ),
    )
    # .persist()
)
print("loaded resolvedColloc")
window_spec = Window.partitionBy("targetId", "diseaseId",'leftStudyId').orderBy( ### include gwas study
    F.col("pValueExponent").asc()
)
gwasCredibleAssoc = (
resolvedColoc.withColumn(
    "homogenized", F.first("colocDoE", ignorenulls=True).over(window_spec)
)  ## added 30.01.2025
.select("targetId", "diseaseId",'leftStudyId', "homogenized")
.withColumn(
    "homogenized",
    F.when(F.col("homogenized").isNull(), F.lit("noEvaluable")).otherwise(
        F.col("homogenized")
    ),
)
)

datasource_filter = [
#"gwas_credible_set", remove so avoid potential duplicates as it will be incorporated later (DoE is done separately)
"gene_burden",
"eva",
"eva_somatic",
"gene2phenotype",
"orphanet",
"cancer_gene_census",
"intogen",
"impc",
"chembl",
]

assessment, evidences, actionType, oncolabel = temporary_directionOfEffect(
path, datasource_filter
)
assessment_all = assessment.unionByName(
gwasCredibleAssoc.withColumn("datasourceId", F.lit("gwas_credible_set")),
allowMissingColumns=True,
)



loaded newColoc


loaded gwasComplete
loaded resolvedColloc


In [2]:
#from DoEAssessment import directionOfEffect
from functions import temporary_directionOfEffect,build_gwasResolvedColoc_noPropag
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F
from pyspark.ml.feature import QuantileDiscretizer
from datetime import date
from doeFunction import doeFunction

spark = SparkSession.builder.getOrCreate()
today_date = str(date.today())

#platform_v = "25.03"
path = 'gs://open-targets-data-releases/25.03/output/'

replacement_dict = {
    "gene_burden": "GeneBurden",
    "chembl": "ChEMBL",
    "intogen": "Intogen",
    "orphanet": "Orphanet",
    "cancer_gene_census": "CancerGeneCensus",
    "eva": "EvaGermline",
    "gene2phenotype": "Gene2Phenotype",
    "eva_somatic": "EvaSomatic",
    "gwas_credible_sets": "OtGenetics",
    "impc": "IMPC",
}

doe_sources = [
    "gwas_credible_sets",
    "gene_burden",
    "eva",
    "eva_somatic",
    "gene2phenotype",
    "orphanet",
    "cancer_gene_census",
    "intogen",
    "impc",
    "chembl",
]

new = spark.read.parquet(f"{path}colocalisation_coloc") 
credible = spark.read.parquet(f"{path}credible_set")
index=spark.read.parquet(f"{path}study/")
evidences = spark.read.parquet(f"{path}evidence")
diseases = spark.read.parquet(f"{path}disease/")

### take only the ones with datasources for DoE
evidences = evidences.filter(F.col("datasourceId").isin(doe_sources))

gwasCredibleAssoc,assessment=doeFunction(new,credible,index,evidences,diseases,path)

gwasCredibleAssoc.withColumn(
        "homogenized",
        F.when(F.col("homogenized").isNull(), F.lit("noEvaluable")).otherwise(
            F.col("homogenized")
        ),
    ).withColumn("variantEffect", 
                F.when(F.col("homogenized").isin(["LoF_protect","LoF_risk"]), F.lit("LoF")
                ).when(F.col("homogenized").isin(["GoF_protect","GoF_risk"]), F.lit("GoF")
                ).otherwise(F.lit("noEvaluable"))
    ).withColumn("directionOnTrait", 
                F.when(F.col("homogenized").isin(["LoF_protect","GoF_protect"]), F.lit("protect")
                ).when(F.col("homogenized").isin(["LoF_risk","GoF_risk"]), F.lit("risk")
                ).otherwise(F.lit("noEvaluable"))
    ).select("targetId", "diseaseId", "homogenized","variantEffect","directionOnTrait")


### replace     
assessment_all = assessment.unionByName(
    gwasCredibleAssoc.withColumn("datasourceId", F.lit("gwas_credible_set")),
    allowMissingColumns=True,
    ).withColumn("niceName", F.col("datasourceId")
    ).replace(replacement_dict, subset=["niceName"]
    ).withColumn("datasourceAll", F.lit("All"))

print("asessment done")

def datasets_numbers_trait(assessment_all, buckets_number):
    """This function creates in a long format (suitable for R) the N of evidences and association per
    DoE section. At the end, it creates a column with the corresponding deciles of the numbers to plot their intesntiy
    The deciles are trained using the assoc to avoid underrating numbers from the assoc respecting the evidences
    """
    """ 
        usage of temporary_directionOfEffect:
        path = "gs://open-targets-pre-data-releases/24.12-uo_test-3/output/etl/parquet/"

        datasource_filter = [
            "ot_genetics_portal",
            "gwas_credible_sets",
            "gene_burden",
            "eva",
            "eva_somatic",
            "gene2phenotype",
            "orphanet",
            "cancer_gene_census",
            "intogen",
            "impc",
            "chembl",
        ]

        assessment, evidences, actionType, oncolabel = temporary_directionOfEffect(
            path, datasource_filter
        )
    """

    ## direction on trait
    unpivot_trait = "stack(2, 'protect', protect, 'risk', risk) as (type, count)"
    trait = (
        assessment_all.filter(F.col("directionOnTrait") != "noEvaluable")
        .groupBy("niceName")
        .pivot("directionOnTrait")
        .count()
        .union(
            assessment_all.filter(F.col("directionOnTrait") != "noEvaluable")
            .groupBy("datasourceAll")
            .pivot("directionOnTrait")
            .count()
            .withColumnRenamed("datasourceAll", "niceName")
        )
        .select("niceName", F.expr(unpivot_trait))
        .fillna(0)
    ).withColumn("facet", F.lit("trait"))

    #### direction on target
    unpivot_function = "stack(2, 'gof', gof, 'lof', lof) as (type, count)"

    function = (
        assessment_all.filter(F.col("variantEffect") != "noEvaluable")
        .groupBy("niceName")
        .pivot("variantEffect")
        .count()
        .union(
            assessment_all.filter(F.col("variantEffect") != "noEvaluable")
            .groupBy("datasourceAll")
            .pivot("variantEffect")
            .count()
            .withColumnRenamed("datasourceAll", "niceName")
        )
        .select("niceName", F.expr(unpivot_function))
        .fillna(0)
    ).withColumn("facet", F.lit("function"))

    #### direction complete
    unpivot_whole = "stack(4, 'LoF_protect', LoF_protect, 'LoF_risk', LoF_risk,'GoF_protect',GoF_protect,'GoF_risk',GoF_risk) as (type, count)"

    whole = (
        assessment_all.filter(F.col("homogenized") != "noEvaluable")
        .groupBy("targetId", "diseaseId", "niceName", "homogenized")
        .count()
        .groupBy("niceName")
        .pivot("homogenized")
        .count()
        .union(
            assessment_all.filter(F.col("homogenized") != "noEvaluable")
            .groupBy("targetId", "diseaseId", "datasourceAll", "homogenized")
            .count()
            .groupBy("datasourceAll")
            .pivot("homogenized")
            .count()
            .withColumnRenamed("datasourceAll", "niceName")
        )
        .select("niceName", F.expr(unpivot_whole))
        .fillna(0)
    ).withColumn("facet", F.lit("whole"))

    qds = QuantileDiscretizer(
        numBuckets=buckets_number,
        inputCol="count",
        outputCol="deciles",
    )

    all = function.union(trait).union(whole)
    result = qds.fit(whole).transform(all)  ### train qds in Whole and transform all
    
    return result

print("read function")

print("executing function")

dataset=datasets_numbers_trait(assessment_all, 11)
print("executed function")
print("printing dataset")
dataset.toPandas().to_csv(f"gs://ot-team/jroldan/analysis/{today_date}_numbersDoeTable.csv")
print("dataset saved succesfully")

loaded newColoc


loaded gwasComplete
loaded resolvedColloc


25/05/10 09:30:06 WARN CacheManager: Asked to cache already cached data.
25/05/10 09:30:07 WARN CacheManager: Asked to cache already cached data.


asessment done
read function
executing function


25/05/10 09:31:40 WARN QuantileDiscretizer: Some quantiles were identical. Bucketing to 7 buckets as a result.


executed function
printing dataset


dataset saved succesfully


In [1]:
import time
from array import ArrayType
from functions import (
    relative_success,
    spreadSheetFormatter,
    discrepancifier,
    temporary_directionOfEffect,
)
# from stoppedTrials import terminated_td
from DoEAssessment import directionOfEffect
# from membraneTargets import target_membrane
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F
from datetime import datetime
from datetime import date
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.types import (
    StructType,
    StructField,
    DoubleType,
    DecimalType,
    StringType,
    FloatType,
)
import pandas as pd

spark = SparkSession.builder.getOrCreate()
spark.conf.set(
    "spark.sql.shuffle.partitions", "400"
)  # Default is 200, increase if needed


path_n='gs://open-targets-data-releases/25.03/output/'

target = spark.read.parquet(f"{path_n}target/")

diseases = spark.read.parquet(f"{path_n}disease/")

evidences = spark.read.parquet(f"{path_n}evidence")

credible = spark.read.parquet(f"{path_n}credible_set")

new = spark.read.parquet(f"{path_n}colocalisation_coloc") 

index=spark.read.parquet(f"{path_n}study/")

variantIndex = spark.read.parquet(f"{path_n}variant")

biosample = spark.read.parquet(f"{path_n}biosample")

print("loaded files")

newColoc = (
    new.join(
        credible.selectExpr(  #### studyLocusId from credible set to uncover the codified variants on left side
            "studyLocusId as leftStudyLocusId",
            "StudyId as leftStudyId",
            "variantId as leftVariantId",
            "studyType as credibleLeftStudyType",
        ),
        on="leftStudyLocusId",
        how="left",
    )
    .join(
        credible.selectExpr(  #### studyLocusId from credible set to uncover the codified variants on right side
            "studyLocusId as rightStudyLocusId",
            "studyId as rightStudyId",
            "variantId as rightVariantId",
            "studyType as credibleRightStudyType",
            'isTransQtl'
        ),
        on="rightStudyLocusId",
        how="left",
    )
    .join(
        index.selectExpr(  ### bring modulated target on right side (QTL study)
            "studyId as rightStudyId",
            "geneId",
            "projectId",
            "studyType as indexStudyType",
            "condition",
            "biosampleId",
        ),
        on="rightStudyId",
        how="left",
)
    # .persist()
)

print("loaded newColoc")

# remove columns without content (only null values on them)
df = evidences.filter((F.col("datasourceId") == "gwas_credible_sets"))

# Use an aggregation to determine non-null columns
non_null_counts = df.select(
    *[F.sum(F.col(col).isNotNull().cast("int")).alias(col) for col in df.columns]
)

# Collect the counts for each column
non_null_columns = [
    row[0] for row in non_null_counts.collect()[0].asDict().items() if row[1] > 0
]

# Select only the non-null columns
filtered_df = df.select(*non_null_columns)  # .persist()

## bring studyId, variantId, beta from Gwas and pValue
gwasComplete = filtered_df.join(
    credible.selectExpr(
        "studyLocusId", "studyId", "variantId", "beta as betaGwas", "pValueExponent"
    ),
    on="studyLocusId",
    how="left",
)  # .persist()

print("loaded gwasComplete")

resolvedColoc = (
    (
        newColoc.withColumnRenamed("geneId", "targetId")
        .join(
            gwasComplete.withColumnRenamed("studyLocusId", "leftStudyLocusId"),
            on=["leftStudyLocusId", "targetId"],
            how="inner",
        )
        .join(  ### propagated using parent terms
            diseases.selectExpr(
                "id as diseaseId", "name", "parents", "therapeuticAreas"
            ),
            on="diseaseId",
            how="left",
        )
        .withColumn(
            "diseaseId",
            F.explode_outer(F.concat(F.array(F.col("diseaseId")), F.col("parents"))),
        )
        .drop("parents", "oldDiseaseId")
    ).withColumn(
        "colocDoE",
        F.when(
            F.col("rightStudyType").isin(
                ["eqtl", "pqtl", "tuqtl", "sceqtl", "sctuqtl"]
            ),
            F.when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("GoF_risk"),
            )
            .when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("LoF_risk"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("LoF_protect"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("GoF_protect"),
            ),
        ).when(
            F.col("rightStudyType").isin(
                ["sqtl", "scsqtl"]
            ),  ### opposite directionality than sqtl
            F.when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("LoF_risk"),
            )
            .when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("GoF_risk"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("GoF_protect"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("LoF_protect"),
            ),
        ),
    )
    # .persist()
)
print("loaded resolvedColloc")

path = "gs://open-targets-pre-data-releases/24.12-uo_test-3/output/etl/parquet/"

datasource_filter = [
    "ot_genetics_portal",
    "gwas_credible_sets",
    "gene_burden",
    "eva",
    "eva_somatic",
    "gene2phenotype",
    "orphanet",
    "cancer_gene_census",
    "intogen",
    "impc",
    "chembl",
]

assessment, evidences, actionType, oncolabel = temporary_directionOfEffect(
    path_n, datasource_filter
)

print("run temporary direction of effect")


print("built drugApproved dataset")

analysis_chembl_indication = (
    discrepancifier(
        assessment.filter((F.col("datasourceId") == "chembl"))
        .withColumn(
            "maxClinPhase",
            F.max(F.col("clinicalPhase")).over(
                Window.partitionBy("targetId", "diseaseId")
            ),
        )
        .groupBy("targetId", "diseaseId", "maxClinPhase")
        .pivot("homogenized")
        .agg(F.count("targetId"))
    )
    .filter(F.col("coherencyDiagonal") == "coherent")
    .drop(
        "coherencyDiagonal", "coherencyOneCell", "noEvaluable", "GoF_risk", "LoF_risk"
    )
    .withColumnRenamed("GoF_protect", "drugGoF_protect")
    .withColumnRenamed("LoF_protect", "drugLoF_protect")
    # .persist()
)

chemblAssoc = (
    discrepancifier(
        assessment.filter(
            (F.col("datasourceId") == "chembl")
            & (F.col("homogenized") != "noEvaluable")
        )
        .withColumn(
            "maxClinPhase",
            F.max("clinicalPhase").over(Window.partitionBy("targetId", "diseaseId")),
        )
        .groupBy("targetId", "diseaseId", "maxClinPhase")
        .pivot("homogenized")
        .count()
    )
    .filter(F.col("coherencyDiagonal") == "coherent")
    .drop(
        "coherencyDiagonal", "coherencyOneCell", "noEvaluable", "GoF_risk", "LoF_risk"
    )
    .withColumnRenamed("GoF_protect", "drugGoF_protect")
    .withColumnRenamed("LoF_protect", "drugLoF_protect")
)

print("built chemblAssoc dataset")

benchmark = (
    (
        resolvedColoc.filter(F.col("betaGwas") < 0)
        .join(  ### select just GWAS giving protection
            analysis_chembl_indication, on=["targetId", "diseaseId"], how="inner"
        )
        .withColumn(
            "AgreeDrug",
            F.when(
                (F.col("drugGoF_protect").isNotNull())
                & (F.col("colocDoE") == "GoF_protect"),
                F.lit("yes"),
            )
            .when(
                (F.col("drugLoF_protect").isNotNull())
                & (F.col("colocDoE") == "LoF_protect"),
                F.lit("yes"),
            )
            .otherwise(F.lit("no")),
        )
    ).filter(
        F.col("name") != "COVID-19"
    )  #### remove COVID-19 associations
).join(biosample.select("biosampleId", "biosampleName"), on="biosampleId", how="left")


print("built benchmark dataset")

#### Analysis

#### 1 Build a dictionary with the distinct values as key and column names as value
variables_study = ["projectId", "biosampleName", "rightStudyType", "colocDoE"]

# Initialize an empty dictionary
disdic = {}

# Iterate over the list of column names
for col_name in variables_study:
    # Extract distinct values for the column
    distinct_values = benchmark.select(col_name).distinct().collect()

    # Populate the dictionary
    for row in distinct_values:
        distinct_value = row[col_name]
        if distinct_value is not None:  # Exclude None (null) values
            disdic[distinct_value] = col_name


spark session created at 2025-05-13 19:11:59.145033
Analysis started on 2025-05-13 at  2025-05-13 19:11:59.145033


25/05/13 19:12:04 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


loaded files
loaded newColoc


loaded gwasComplete
loaded resolvedColloc
run temporary direction of effect
built drugApproved dataset


built chemblAssoc dataset
built benchmark dataset


In [2]:
disdic

{'HipSci': 'projectId',
 'van_de_Bunt_2015': 'projectId',
 'BrainSeq': 'projectId',
 'GTEx': 'projectId',
 'Bossini-Castillo_2019': 'projectId',
 'Jerber_2021': 'projectId',
 'ROSMAP': 'projectId',
 'TwinsUK': 'projectId',
 'GEUVADIS': 'projectId',
 'FUSION': 'projectId',
 'CommonMind': 'projectId',
 'PhLiPS': 'projectId',
 'Aygun_2021': 'projectId',
 'OneK1K': 'projectId',
 'UKB_PPP_EUR': 'projectId',
 'Walker_2019': 'projectId',
 'Schmiedel_2018': 'projectId',
 'BLUEPRINT': 'projectId',
 'Cytoimmgen': 'projectId',
 'Quach_2016': 'projectId',
 'CEDAR': 'projectId',
 'PISA': 'projectId',
 'Lepik_2017': 'projectId',
 'Alasoo_2018': 'projectId',
 'GENCORD': 'projectId',
 'Sun_2018': 'projectId',
 'Peng_2018': 'projectId',
 'iPSCORE': 'projectId',
 'Gilchrist_2021': 'projectId',
 'Braineac2': 'projectId',
 'Nedelec_2016': 'projectId',
 'Nathan_2022': 'projectId',
 'Perez_2022': 'projectId',
 'CAP': 'projectId',
 'Young_2019': 'projectId',
 'Schwartzentruber_2018': 'projectId',
 'Fairfax_2

In [4]:
dataset.show()

+---+---------+--------------------+------+---------+-------------------+-------------+-------------+-----+--------------------+----------+-------+-------+--------------------+-----------+------------------+-------+-------+-------+-------+---------+-----------+----------------+-------+--------------------+------+
|_c0|    group|          comparison| phase|oddsRatio|             pValue|lowerInterval|upperInterval|total|              values|relSuccess|rsLower|rsUpper|                path|significant|       writeFigure|value_1|value_2|value_3|value_4|numerator|denominator|pValue_formatted| numDen|              prefix|suffix|
+---+---------+--------------------+------+---------+-------------------+-------------+-------------+-----+--------------------+----------+-------+-------+--------------------+-----------+------------------+-------+-------+-------+-------+---------+-----------+----------------+-------+--------------------+------+
|  0|predictor|         HipSci_only|Phase4|     0.61| 0

In [3]:
### read file and annotate to which element of dictionary belong to 

dataset=spark.read.csv('gs://ot-team/jroldan/analysis/2025-04-09_credibleSetColocDoEanalysis_RightTissues.csv', header=True)



In [5]:
from pyspark.sql.functions import create_map
from itertools import chain

mapping_expr=create_map([F.lit(x) for x in chain(*disdic.items())])

In [7]:
dataset.show()

+---+---------+--------------------+------+---------+-------------------+-------------+-------------+-----+--------------------+----------+-------+-------+--------------------+-----------+------------------+-------+-------+-------+-------+---------+-----------+----------------+-------+--------------------+------+
|_c0|    group|          comparison| phase|oddsRatio|             pValue|lowerInterval|upperInterval|total|              values|relSuccess|rsLower|rsUpper|                path|significant|       writeFigure|value_1|value_2|value_3|value_4|numerator|denominator|pValue_formatted| numDen|              prefix|suffix|
+---+---------+--------------------+------+---------+-------------------+-------------+-------------+-----+--------------------+----------+-------+-------+--------------------+-----------+------------------+-------+-------+-------+-------+---------+-----------+----------------+-------+--------------------+------+
|  0|predictor|         HipSci_only|Phase4|     0.61| 0

In [10]:
dataset_wValues.groupBy('annotation').count().show()

+--------------+-----+
|    annotation|count|
+--------------+-----+
|      colocDoE|    4|
| biosampleName|  158|
|     projectId|   78|
|rightStudyType|   10|
+--------------+-----+



In [8]:
dataset_wValues=dataset.withColumn('annotation',mapping_expr.getItem(F.col('prefix')))

/usr/lib/spark/python/pyspark/sql/column.py:460: FutureWarning: A column as 'key' in getItem is deprecated as of Spark 3.0, and will not be supported in the future release. Use `column[key]` or `column.key` syntax instead.
  warnings.warn(


In [11]:
dataset_wValues.toPandas().to_csv('gs://ot-team/jroldan/analysis/2025-04-09_credibleSetColocDoEanalysis_RightTissues_annotated.csv')

In [ ]:
#### check why the numbers from rightTissue are so similar: 



##### 


In [12]:
#### BUILDING THE NEW GWAS GENETIC EVIDENCE FROM COLOC
from functions import (
    discrepancifier,
    build_gwasResolvedColoc,
    temporary_directionOfEffect,
    spreadSheetFormatter,
)
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F
import matplotlib.pyplot as plt
from datetime import date, datetime
from pyspark.sql.types import (
    StructType,
    StructField,
    ArrayType,
    DoubleType,
    DecimalType,
    StringType,
    FloatType,
    IntegerType,
)
import pandas as pd

spark = SparkSession.builder.getOrCreate()

# path = "gs://open-targets-pre-data-releases/24.12-uo_test-3/output/etl/parquet/"

path_n='gs://open-targets-data-releases/25.03/output/'


evidences = (
    spark.read.parquet(f"{path_n}evidence").filter(
        F.col("datasourceId").isin(
            [
                "gwas_credible_sets",
                "chembl",
            ]
        )
    )
    # .persist()
)

negativeTD = (
    evidences.filter(F.col("datasourceId") == "chembl")
    .select("targetId", "diseaseId", "studyStopReason", "studyStopReasonCategories")
    .filter(F.array_contains(F.col("studyStopReasonCategories"), "Negative"))
    .groupBy("targetId", "diseaseId")
    .count()
    .withColumn("stopReason", F.lit("Negative"))
    .drop("count")
)

#### include here right way of generating gwasResolvedColoc
#### resolved coloc: 
diseases = spark.read.parquet(f"{path_n}disease/")

credible = spark.read.parquet(f"{path_n}credible_set")

new = spark.read.parquet(f"{path_n}colocalisation_coloc") 

index=spark.read.parquet(f"{path_n}study/")

variantIndex = spark.read.parquet(f"{path_n}variant")

biosample = spark.read.parquet(f"{path_n}biosample")

print("loaded files")

newColoc = (
    new.join(
        credible.selectExpr(  #### studyLocusId from credible set to uncover the codified variants on left side
            "studyLocusId as leftStudyLocusId",
            "StudyId as leftStudyId",
            "variantId as leftVariantId",
            "studyType as credibleLeftStudyType",
        ),
        on="leftStudyLocusId",
        how="left",
    )
    .join(
        credible.selectExpr(  #### studyLocusId from credible set to uncover the codified variants on right side
            "studyLocusId as rightStudyLocusId",
            "studyId as rightStudyId",
            "variantId as rightVariantId",
            "studyType as credibleRightStudyType",
            'isTransQtl'
        ),
        on="rightStudyLocusId",
        how="left",
    )
    .join(
        index.selectExpr(  ### bring modulated target on right side (QTL study)
            "studyId as rightStudyId",
            "geneId",
            "projectId",
            "studyType as indexStudyType",
            "condition",
            "biosampleId",
        ),
        on="rightStudyId",
        how="left",
)
    # .persist()
)

print("loaded newColoc")

# remove columns without content (only null values on them)
df = evidences.filter((F.col("datasourceId") == "gwas_credible_sets"))

# Use an aggregation to determine non-null columns
non_null_counts = df.select(
    *[F.sum(F.col(col).isNotNull().cast("int")).alias(col) for col in df.columns]
)

# Collect the counts for each column
non_null_columns = [
    row[0] for row in non_null_counts.collect()[0].asDict().items() if row[1] > 0
]

# Select only the non-null columns
filtered_df = df.select(*non_null_columns)  # .persist()

## bring studyId, variantId, beta from Gwas and pValue
gwasComplete = filtered_df.join(
    credible.selectExpr(
        "studyLocusId", "studyId", "variantId", "beta as betaGwas", "pValueExponent"
    ),
    on="studyLocusId",
    how="left",
)  # .persist()

print("loaded gwasComplete")

resolvedColoc = (
    (
        newColoc.withColumnRenamed("geneId", "targetId")
        .join(
            gwasComplete.withColumnRenamed("studyLocusId", "leftStudyLocusId"),
            on=["leftStudyLocusId", "targetId"],
            how="right", ### has to be right 
        )
#        .join(  ### propagated using parent terms  ### no propagation yet 
#            diseases.selectExpr(
#                "id as diseaseId", "name", "parents", "therapeuticAreas"
#            ),
#            on="diseaseId",
#            how="left",
#        )
#        .withColumn(
#            "diseaseId",
#            F.explode_outer(F.concat(F.array(F.col("diseaseId")), F.col("parents"))),
#        )
#        .drop("parents", "oldDiseaseId")
    ).withColumn(
        "colocDoE",
        F.when(
            F.col("rightStudyType").isin(
                ["eqtl", "pqtl", "tuqtl", "sceqtl", "sctuqtl"]
            ),
            F.when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("GoF_risk"),
            )
            .when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("LoF_risk"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("LoF_protect"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("GoF_protect"),
            ),
        ).when(
            F.col("rightStudyType").isin(
                ["sqtl", "scsqtl"]
            ),  ### opposite directionality than sqtl
            F.when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("LoF_risk"),
            )
            .when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("GoF_risk"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("GoF_protect"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("LoF_protect"),
            ),
        ),
    )
    # .persist()
)
print("loaded resolvedColloc")

##### 


### alternative for gwas credible assoc 
window_spec = Window.partitionBy("targetId", "diseaseId",'leftStudyId').orderBy( ### include gwas study
    F.col("pValueExponent").asc()
)
gwasCredibleAssoc = (
    resolvedColoc.withColumn(
        "homogenized", F.first("colocDoE", ignorenulls=True).over(window_spec)
    )  ## added 30.01.2025
    .select(
        "targetId",
        "diseaseId",
        "homogenized",
        "leftStudyLocusId",
        "h4",
        "datasourceId",
        "resourceScore",
        "leftVariantId",
        "credibleLeftStudyType",
    )
    .withColumn(
        "homogenized",
        F.when(F.col("homogenized").isNull(), F.lit("noEvaluable")).otherwise(
            F.col("homogenized")
        ),
    )
)


#### LOAD STUDYLOCUSID AND VARIANT DISTANCES

l2gPred=spark.read.parquet(f"{path_n}l2g_prediction/")

l2gTable = (
    l2gPred.select("studyLocusId", "geneId", F.explode_outer("features")).select('studyLocusId','geneId','col.name','col.value').filter(
        F.col("col.name").isin(["distanceFootprintMean", "distanceTssMean"])
    )
    .groupBy("studyLocusId", "geneId")
    .pivot("name")
    .agg(F.first("value"))
)

print("creating gwasCredibleAssocDistances")
gwasCredibleAssocDistances = gwasCredibleAssoc.join(
    l2gTable.withColumnRenamed("studyLocusId", "leftStudyLocusId").withColumnRenamed(
        "geneId", "targetId"
    ),
    on=["leftStudyLocusId", "targetId"],
    how="left",
)

print("creating analysis_chembl")

datasource_filter = ["chembl"]
preanalysis_chembl, evidences, actionType, oncolabel = temporary_directionOfEffect(
    path_n, datasource_filter
)
analysis_chembl = (
    discrepancifier(
        preanalysis_chembl.withColumn(
            "maxClinPhase",
            F.max(F.col("clinicalPhase")).over(
                Window.partitionBy("targetId", "diseaseId")
            ),
        )
        .groupBy("targetId", "diseaseId", "maxClinPhase")
        .pivot("homogenized")
        .agg(F.count("targetId"))
        .persist()
    )  #### avoid the
    #    .filter(  ### ensure drug has annotated MoA and is coherent per Target-Disease
    #        ((F.col("GoF_protect").isNotNull()) | F.col("LoF_protect").isNotNull())
    #        & (F.col("coherencyDiagonal") == "coherent")
).selectExpr(
    "targetId",
    "diseaseId",
    "maxClinPhase",
    "coherencyDiagonal as coherencyDiagonal_ch",
    "coherencyOneCell as coherencyOneCell_ch",
    "LoF_protect as LoF_protect_ch",
    "GoF_protect as GoF_protect_ch",
)

### NEWWWWW ### 
doe_columns=["LoF_protect", "GoF_risk", "LoF_risk", "GoF_protect"]
diagonal_lof=['LoF_protect','GoF_risk']
diagonal_gof=['LoF_risk','GoF_protect']

diseases2 = diseases.select("id", "parents").withColumn(
    "diseaseIdPropagated",
    F.explode_outer(F.concat(F.array(F.col("id")), F.col("parents"))),
)
### pivot colocdoE grouping by T-D-studyLocusId-distances

value = "max_L2GScore"

list_l2g = [
    0.10,
    0.15,
    0.20,
    0.25,
    0.30,
    0.35,
    0.40,
    0.45,
    0.50,
    0.55,
    0.60,
    0.65,
    0.70,
    0.72,
    0.74,
    0.75,
    0.76,
    0.78,
    0.80,
    0.82,
    0.84,
    0.85,
    0.86,
    0.88,
    0.90,
    0.92,
    0.94,
    0.95,
    0.96,
    0.98,
]
print("creating benchmarkOT function")

dict_comb = {}


def benchmarkOT(
    dict_comb,
    value,
    gwasCredibleAssocDistances,
    analysis_chembl,
    negativeTD,
    list_l2g,
):
    dict_comb = {
        "hasGeneticEvidence": f"{value}",
        "diagonalYes": f"{value}",
        "oneCellYes": f"{value}",
        "L2GAndColoc": f"{value}",
        'NoneCellYes': f"{value}",
        'NdiagonalYes': f"{value}",
    }
    min_value = gwasCredibleAssocDistances.agg(
        F.min("distanceFootprintMean")
    ).collect()[0][
        0
    ]  ## take min value for scalating
    conditions = [
        F.when(F.col(c) == F.col("maxDoE"), F.lit(c)).otherwise(F.lit(None)) for c in doe_columns
        ]

    dataset_original = discrepancifier(
        gwasCredibleAssocDistances
        # .filter(F.col("h4").isNotNull()) #### not filter by this because we want to include the L2G AND Coloc question
        .withColumn(  ### take maximum L2G score per T-D
            "max_L2GScore",
            F.max("resourceScore").over(Window.partitionBy("targetId", "diseaseId")),
        )
        .withColumn("min_distanceFootprintMean", F.lit(min_value))
        .withColumn(
            "scaled_distanceFootprintMean",
            (F.col("distanceFootprintMean") - F.col("min_distanceFootprintMean"))
            / (1 - F.col("min_distanceFootprintMean")),
        )
        .withColumn(
            "minDistFootPrintMean",
            F.min("scaled_distanceFootprintMean").over(
                Window.partitionBy("targetId", "diseaseId")
            ),
        )
        .withColumn(
            "minTssDistance",
            F.min("distanceTssMean").over(Window.partitionBy("targetId", "diseaseId")),
        )
        .groupBy(
            "targetId",
            "diseaseId",
            f"{value}",
            # "leftStudyLocusId",
        )
        .pivot("homogenized")
        .count()
    )
    dataset_propag = discrepancifier(
        gwasCredibleAssocDistances
        # .filter(F.col("h4").isNotNull()) #### not filter by this because we want to include the L2G AND Coloc question
        .withColumn(  ### take maximum L2G score per T-D
            "max_L2GScore",
            F.max("resourceScore").over(Window.partitionBy("targetId", "diseaseId")),
        )
        .withColumn("min_distanceFootprintMean", F.lit(min_value))
        .withColumn(
            "scaled_distanceFootprintMean",
            (F.col("distanceFootprintMean") - F.col("min_distanceFootprintMean"))
            / (1 - F.col("min_distanceFootprintMean")),
        )
        .withColumn(
            "minDistFootPrintMean",
            F.min("scaled_distanceFootprintMean").over(
                Window.partitionBy("targetId", "diseaseId")
            ),
        )
        .withColumn(
            "minTssDistance",
            F.min("distanceTssMean").over(Window.partitionBy("targetId", "diseaseId")),
        )
        .join(
            diseases2.selectExpr("id as diseaseId", "diseaseIdPropagated"),
            on="diseaseId",
            how="left",
        )
        .withColumnRenamed("diseaseId", "oldDiseaseId")
        .withColumnRenamed("diseaseIdPropagated", "diseaseId")
        .groupBy(
            "targetId",
            "diseaseId",
            f"{value}",
            # "leftStudyLocusId",
        )
        .pivot("homogenized")
        .count()
    )

    return (
        dataset_original.join(
            analysis_chembl, on=["targetId", "diseaseId"], how="right"
        )
        .withColumn(
            "diagonalAgreeWithDrugs",
            F.when(
                (
                    (F.col("coherencyDiagonal_ch") == "coherent")
                    & (F.col("coherencyDiagonal") == "coherent")
                )
                # & (F.col("coherencyDiagonal") == "coherent")
                ,
                F.when(
                    (F.col("LoF_protect_ch").isNotNull())
                    & (
                        F.col("GoF_risk").isNotNull() | F.col("LoF_protect").isNotNull()
                    ),
                    F.lit("coherent"),
                )
                .when(
                    F.col("GoF_protect_ch").isNotNull()
                    & (
                        F.col("LoF_risk").isNotNull() | F.col("GoF_protect").isNotNull()
                    ),
                    F.lit("coherent"),
                )
                .otherwise(F.lit("dispar")),
            ),
        )
        .withColumn(
            "oneCellAgreeWithDrugs",
            F.when(
                (
                    (F.col("coherencyOneCell_ch") == "coherent")
                    & (F.col("coherencyDiagonal") == "coherent")
                )
                # & (F.col("coherencyOneCell") == "coherent")
                ,
                F.when(
                    (F.col("LoF_protect_ch").isNotNull())
                    & (
                        (F.col("LoF_protect").isNotNull())
                        & (F.col("LoF_risk").isNull())
                        & (F.col("GoF_protect").isNull())
                        & (F.col("GoF_risk").isNull())
                    ),
                    F.lit("coherent"),
                )
                .when(
                    (F.col("GoF_protect_ch").isNotNull())
                    & (
                        (F.col("GoF_protect").isNotNull())
                        & (F.col("LoF_risk").isNull())
                        & (F.col("LoF_protect").isNull())
                        & (F.col("GoF_risk").isNull())
                    ),
                    F.lit("coherent"),
                )
                .otherwise(F.lit("dispar")),
            ),
            # ).filter(
            #    F.col("diagonalAgreeWithDrugs").isNotNull()
        ).withColumn(
            "arrayN", F.array(*[F.col(c) for c in doe_columns])
        ).withColumn(
            "maxDoE", F.array_max(F.col("arrayN"))
        ).withColumn("maxDoE_names", F.array(*conditions)
        ).withColumn("maxDoE_names", F.expr("filter(maxDoE_names, x -> x is not null)")
        )
        .withColumn(
            "Phase4",
            F.when(F.col("maxClinPhase") == 4, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "Phase>=3",
            F.when(F.col("maxClinPhase") >= 3, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "Phase>=2",
            F.when(F.col("maxClinPhase") >= 2, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "Phase>=1",
            F.when(F.col("maxClinPhase") >= 1, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "Phase0",
            F.when(F.col("maxClinPhase") == 0, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "diagonalYes",
            F.when(
                F.col("diagonalAgreeWithDrugs") == "coherent", F.lit("yes")
            ).otherwise(F.lit("no")),
        )
        .withColumn(
            "oneCellYes",
            F.when(
                F.col("oneCellAgreeWithDrugs") == "coherent", F.lit("yes")
            ).otherwise(F.lit("no")),
        )
        .withColumn(
            "hasGeneticEvidence",
            F.when(F.col(f"{value}").isNotNull(), F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "L2GAndColoc",
            F.when(
                (F.col(f"{value}").isNotNull())
                & (F.col("coherencyDiagonal").isin(["coherent", "dispar"])),
                F.lit("yes"),
            ).otherwise(F.lit("no")),
        )
        .withColumn(
            "maxDoEArrayN",
            F.expr("aggregate(arrayN, 0, (acc, x) -> acc + IF(x = maxDoE, 1, 0))")
        ).withColumn(
            "NoneCellYes",
            F.when((F.col("LoF_protect_ch").isNotNull() & (F.col('GoF_protect_ch').isNull())) & (F.array_contains(F.col("maxDoE_names"), F.lit("LoF_protect")))==True, F.lit('yes'))
            .when((F.col("GoF_protect_ch").isNotNull() & (F.col('LoF_protect_ch').isNull())) & (F.array_contains(F.col("maxDoE_names"), F.lit("GoF_protect")))==True, F.lit('yes')
                ).otherwise(F.lit('no'))  # If the value is null, return null # Otherwise, check if name is in array
        ).withColumn(
            "NdiagonalYes",
            F.when((F.col("LoF_protect_ch").isNotNull() & (F.col('GoF_protect_ch').isNull())) & 
                (F.size(F.array_intersect(F.col("maxDoE_names"), F.array([F.lit(x) for x in diagonal_lof]))) > 0),
                F.lit("yes")
            ).when((F.col("GoF_protect_ch").isNotNull() & (F.col('LoF_protect_ch').isNull())) & 
                (F.size(F.array_intersect(F.col("maxDoE_names"), F.array([F.lit(x) for x in diagonal_gof]))) > 0),
                F.lit("yes")
            ).otherwise(F.lit('no'))
        )
        .join(negativeTD, on=["targetId", "diseaseId"], how="left")
        .withColumn(
            "PhaseT",
            F.when(F.col("stopReason") == "Negative", F.lit("yes")).otherwise(F.lit("no")),
        )
        .select(
            ["*"]
            + (
                [  ### single columns
                    F.when(F.col(f"{value}") >= n, F.lit("yes"))
                    .otherwise(F.lit("no"))
                    .alias(f"{value}>={str(n).replace('.', '_')}")
                    for n in list_l2g
                ]
            )
            + (
                [  ### column combinations for Yes/No colums Plus has DoE (any agreement)
                    F.when((F.col(a) == "yes") & (F.col(x) >= n), F.lit("yes"))
                    .otherwise(F.lit("no"))
                    .alias(f"{x}>={str(n).replace('.', '_')}&{a}_combined")
                    for a, x in dict_comb.items()
                    for n in list_l2g
                ]
            )
        ),
        dataset_propag.join(
            analysis_chembl, on=["targetId", "diseaseId"], how="right"
        )
        .withColumn(
            "diagonalAgreeWithDrugs",
            F.when(
                (
                    (F.col("coherencyDiagonal_ch") == "coherent")
                    & (F.col("coherencyDiagonal") == "coherent")
                )
                # & (F.col("coherencyDiagonal") == "coherent")
                ,
                F.when(
                    (F.col("LoF_protect_ch").isNotNull())
                    & (
                        F.col("GoF_risk").isNotNull() | F.col("LoF_protect").isNotNull()
                    ),
                    F.lit("coherent"),
                )
                .when(
                    F.col("GoF_protect_ch").isNotNull()
                    & (
                        F.col("LoF_risk").isNotNull() | F.col("GoF_protect").isNotNull()
                    ),
                    F.lit("coherent"),
                )
                .otherwise(F.lit("dispar")),
            ),
        )
        .withColumn(
            "oneCellAgreeWithDrugs",
            F.when(
                (
                    (F.col("coherencyOneCell_ch") == "coherent")
                    & (F.col("coherencyDiagonal") == "coherent")
                )
                # & (F.col("coherencyOneCell") == "coherent")
                ,
                F.when(
                    (F.col("LoF_protect_ch").isNotNull())
                    & (
                        (F.col("LoF_protect").isNotNull())
                        & (F.col("LoF_risk").isNull())
                        & (F.col("GoF_protect").isNull())
                        & (F.col("GoF_risk").isNull())
                    ),
                    F.lit("coherent"),
                )
                .when(
                    (F.col("GoF_protect_ch").isNotNull())
                    & (
                        (F.col("GoF_protect").isNotNull())
                        & (F.col("LoF_risk").isNull())
                        & (F.col("LoF_protect").isNull())
                        & (F.col("GoF_risk").isNull())
                    ),
                    F.lit("coherent"),
                )
                .otherwise(F.lit("dispar")),
            ),
            # ).filter(
            #    F.col("diagonalAgreeWithDrugs").isNotNull()
        ).withColumn(
            "arrayN", F.array(*[F.col(c) for c in doe_columns])
        ).withColumn(
            "maxDoE", F.array_max(F.col("arrayN"))
        ).withColumn("maxDoE_names", F.array(*conditions)
        ).withColumn("maxDoE_names", F.expr("filter(maxDoE_names, x -> x is not null)")
        )
        .withColumn(
            "Phase4",
            F.when(F.col("maxClinPhase") == 4, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "Phase>=3",
            F.when(F.col("maxClinPhase") >= 3, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "Phase>=2",
            F.when(F.col("maxClinPhase") >= 2, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "Phase>=1",
            F.when(F.col("maxClinPhase") >= 1, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "Phase0",
            F.when(F.col("maxClinPhase") == 0, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "diagonalYes",
            F.when(
                F.col("diagonalAgreeWithDrugs") == "coherent", F.lit("yes")
            ).otherwise(F.lit("no")),
        )
        .withColumn(
            "oneCellYes",
            F.when(
                F.col("oneCellAgreeWithDrugs") == "coherent", F.lit("yes")
            ).otherwise(F.lit("no")),
        )
        .withColumn(
            "hasGeneticEvidence",
            F.when(F.col(f"{value}").isNotNull(), F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "L2GAndColoc",
            F.when(
                (F.col(f"{value}").isNotNull())
                & (F.col("coherencyDiagonal").isin(["coherent", "dispar"])),
                F.lit("yes"),
            ).otherwise(F.lit("no")),
        )
        .withColumn(
            "maxDoEArrayN",
            F.expr("aggregate(arrayN, 0, (acc, x) -> acc + IF(x = maxDoE, 1, 0))")
        ).withColumn(
            "NoneCellYes",
            F.when((F.col("LoF_protect_ch").isNotNull() & (F.col('GoF_protect_ch').isNull())) & (F.array_contains(F.col("maxDoE_names"), F.lit("LoF_protect")))==True, F.lit('yes'))
            .when((F.col("GoF_protect_ch").isNotNull() & (F.col('LoF_protect_ch').isNull())) & (F.array_contains(F.col("maxDoE_names"), F.lit("GoF_protect")))==True, F.lit('yes')
                ).otherwise(F.lit('no'))  # If the value is null, return null # Otherwise, check if name is in array
        ).withColumn(
            "NdiagonalYes",
            F.when((F.col("LoF_protect_ch").isNotNull() & (F.col('GoF_protect_ch').isNull())) & 
                (F.size(F.array_intersect(F.col("maxDoE_names"), F.array([F.lit(x) for x in diagonal_lof]))) > 0),
                F.lit("yes")
            ).when((F.col("GoF_protect_ch").isNotNull() & (F.col('LoF_protect_ch').isNull())) & 
                (F.size(F.array_intersect(F.col("maxDoE_names"), F.array([F.lit(x) for x in diagonal_gof]))) > 0),
                F.lit("yes")
            ).otherwise(F.lit('no'))
        )
        .join(negativeTD, on=["targetId", "diseaseId"], how="left")
        .withColumn(
            "PhaseT",
            F.when(F.col("stopReason") == "Negative", F.lit("yes")).otherwise(F.lit("no")),
        )
        .select(
            ["*"]
            + (
                [  ### single columns
                    F.when(F.col(f"{value}") >= n, F.lit("yes"))
                    .otherwise(F.lit("no"))
                    .alias(f"{value}>={str(n).replace('.', '_')}")
                    for n in list_l2g
                ]
            )
            + (
                [  ### column combinations for Yes/No colums Plus has DoE (any agreement)
                    F.when((F.col(a) == "yes") & (F.col(x) >= n), F.lit("yes"))
                    .otherwise(F.lit("no"))
                    .alias(f"{x}>={str(n).replace('.', '_')}&{a}_combined")
                    for a, x in dict_comb.items()
                    for n in list_l2g
                ]
            )
        )
    )

print("creating dataframes in loop")

values = [
    "max_L2GScore",
    "minDistFootPrintMean",
    "minTssDistance",
]

datasetDict = {}
datasetDict_propag = {}

for value in values:
    if value == "max_L2GScore":
        (
            datasetDict[f"max_L2GScore_original"],
            datasetDict_propag[f"max_L2GScore_propag"],
        ) = benchmarkOT(
            dict_comb,
            value,
            gwasCredibleAssocDistances,
            analysis_chembl,
            negativeTD,
            list_l2g,
        )
    else:
        datasetDict[f"{value}_original"], datasetDict_propag[f"{value}_propag"] = (
            benchmarkOT(
                dict_comb,
                value,
                gwasCredibleAssocDistances,
                analysis_chembl,
                negativeTD,
                list_l2g,
            )
        )


loaded files
loaded newColoc


loaded gwasComplete
loaded resolvedColloc


creating gwasCredibleAssocDistances
creating analysis_chembl


25/05/15 13:51:58 WARN CacheManager: Asked to cache already cached data.
25/05/15 13:51:59 WARN CacheManager: Asked to cache already cached data.
25/05/15 13:51:59 WARN CacheManager: Asked to cache already cached data.
25/05/15 13:51:59 WARN CacheManager: Asked to cache already cached data.
25/05/15 13:51:59 WARN CacheManager: Asked to cache already cached data.


creating benchmarkOT function
creating dataframes in loop


In [16]:
gwasCredibleAssoc.groupBy('homogenized').count().show()


+-----------+-------+
|homogenized|  count|
+-----------+-------+
|GoF_protect|3915592|
|   GoF_risk|4004481|
|LoF_protect|3552434|
|   LoF_risk|3660689|
+-----------+-------+



In [15]:
### are there nulls values?

gwasCredibleAssoc2.groupBy('homogenized').count().show()


+-----------+-------+
|homogenized|  count|
+-----------+-------+
|GoF_protect|3847464|
|   GoF_risk|3917917|
|LoF_protect|3630724|
|   LoF_risk|3737091|
|noEvaluable|1916304|
+-----------+-------+



In [13]:
#### why genetic evience and genetic evidence + coloc has the same numbers??
## I suspect we are being restrictives with genetic evidence 

In [4]:
datasetDict[f"max_L2GScore_original"].filter(F.col('hasGeneticEvidence')=='yes').show()

+---------------+-------------+-------------------+-----------+--------+-----------+--------+-----------+-----------------+----------------+------------+--------------------+-------------------+--------------+--------------+----------------------+---------------------+--------------------+------+-------------+------+--------+--------+--------+------+-----------+----------+------------------+-----------+------------+-----------+------------+----------+------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+---------------

In [ ]:
### cases where DoE from Chembl is NULL
datasetDict[f"max_L2GScore_original"].filter(
    (F.col("hasGeneticEvidence") == "yes")
    & ((F.col("LoF_protect_ch").isNull()) & (F.col("GoF_protect_ch").isNull()))
).count()

28

In [11]:
datasetDict[f"max_L2GScore_original"].filter(F.col('coherencyDiagonal')=='EvidNotDoE').show()

+---------------+-------------+-------------------+-----------+--------+-----------+--------+-----------+-----------------+----------------+------------+--------------------+-------------------+--------------+--------------+----------------------+---------------------+--------------------+------+------------+------+--------+--------+--------+------+-----------+----------+------------------+-----------+------------+-----------+------------+----------+------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+----------------

In [10]:
datasetDict[f"max_L2GScore_original"].withColumn(
    "LoF_protect_chN",
    F.when(F.col("LoF_protect_ch").isNotNull(), F.lit("yes")).otherwise(F.lit(None)),
).withColumn(
    "GoF_protect_chN",
    F.when(F.col("GoF_protect_ch").isNotNull(), F.lit("yes")).otherwise(F.lit(None)),
).groupBy(
    "coherencyDiagonal",
    "LoF_protect_chN",
    "GoF_protect_chN",
    "diagonalAgreeWithDrugs",
    "hasGeneticEvidence",
    'NdiagonalYes',
    'diagonalYes',
).count().show()

+-----------------+---------------+---------------+----------------------+------------------+------------+-----------+-----+
|coherencyDiagonal|LoF_protect_chN|GoF_protect_chN|diagonalAgreeWithDrugs|hasGeneticEvidence|NdiagonalYes|diagonalYes|count|
+-----------------+---------------+---------------+----------------------+------------------+------------+-----------+-----+
|             NULL|            yes|            yes|                  NULL|                no|          no|         no| 1343|
|       EvidNotDoE|            yes|           NULL|                  NULL|               yes|          no|         no|  368|
|             NULL|            yes|           NULL|                  NULL|                no|          no|         no|57319|
|             NULL|           NULL|            yes|                  NULL|                no|          no|         no|10690|
|             NULL|           NULL|           NULL|                  NULL|                no|          no|         no| 4067|


In [16]:
### fixed nDoE when drug is dispar
datasetDict[f"max_L2GScore_original"].withColumn(
    "LoF_protect_chN",
    F.when(F.col("LoF_protect_ch").isNotNull(), F.lit("yes")).otherwise(F.lit(None)),
).withColumn(
    "GoF_protect_chN",
    F.when(F.col("GoF_protect_ch").isNotNull(), F.lit("yes")).otherwise(F.lit(None)),
).groupBy(
    "coherencyDiagonal",
    "LoF_protect_chN",
    "GoF_protect_chN",
    "diagonalAgreeWithDrugs",
    #"hasGeneticEvidence",
    'NdiagonalYes',
    'diagonalYes',
).count().show()

+-----------------+---------------+---------------+----------------------+------------+-----------+-----+
|coherencyDiagonal|LoF_protect_chN|GoF_protect_chN|diagonalAgreeWithDrugs|NdiagonalYes|diagonalYes|count|
+-----------------+---------------+---------------+----------------------+------------+-----------+-----+
|             NULL|            yes|            yes|                  NULL|          no|         no| 1343|
|             NULL|           NULL|           NULL|                  NULL|          no|         no| 4067|
|       EvidNotDoE|            yes|           NULL|                  NULL|          no|         no|  368|
|             NULL|            yes|           NULL|                  NULL|          no|         no|57319|
|             NULL|           NULL|            yes|                  NULL|          no|         no|10690|
|         coherent|            yes|            yes|                  NULL|          no|         no|    7|
|       EvidNotDoE|           NULL|           

In [3]:
datasetDict[f"max_L2GScore_original"].filter(
    ((F.col("LoF_protect_ch").isNull()) & (F.col("GoF_protect_ch").isNull()))
).show()

+---------------+-------------+------------+-----------+--------+-----------+--------+-----------+-----------------+----------------+------------+--------------------+-------------------+--------------+--------------+----------------------+---------------------+--------------------+------+------------+------+--------+--------+--------+------+-----------+----------+------------------+-----------+------------+-----------+------------+----------+------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+----

In [9]:
datasetDict[f"max_L2GScore_original"].filter(F.col('noEvaluable').isNotNull()).show()

+--------+---------+------------+-----------+--------+-----------+--------+-----------+-----------------+----------------+------------+--------------------+-------------------+--------------+--------------+----------------------+---------------------+------+------+------------+------+--------+--------+--------+------+-----------+----------+------------------+-----------+------------+-----------+------------+----------+------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+----------

In [ ]:
datasetDict[f"max_L2GScore_original"].groupBy(
    "hasGeneticEvidence", "L2GAndColoc", "coherencyDiagonal",
).count().show()

+------------------+-----------+-----------------+-----+
|hasGeneticEvidence|L2GAndColoc|coherencyDiagonal|count|
+------------------+-----------+-----------------+-----+
|                no|         no|             NULL|73855|
|               yes|        yes|         coherent|  301|
|               yes|        yes|           dispar|   41|
+------------------+-----------+-----------------+-----+



In [ ]:
### new 
datasetDict[f"max_L2GScore_original"].groupBy(
    "hasGeneticEvidence", "L2GAndColoc", "coherencyDiagonal",
).count().show()